# Preparação

## Imports

In [1]:
import os
import duckdb
%load_ext sql

from dotenv import load_dotenv

from pathlib import Path

load_dotenv()

DB_PATH = Path(os.getenv("SILVER_DUCKDB_PATH", ""))
DB_PATH.parent.mkdir(parents=True, exist_ok=True)
TMP_DIR = "/tmp/duckdb_tmp"  # make sure it exists

conn = duckdb.connect(DB_PATH)

conn.execute(f"SET temp_directory='{TMP_DIR}'")
conn.execute("SET memory_limit='64GB'")
conn.execute("SET threads=1")
conn.execute("SET preserve_insertion_order=false")

%sql conn --alias duck

Tip: You may define configurations in /home/c-cc2016/ra173608/tramita/pyproject.toml or /home/c-cc2016/ra173608/.jupysql/config.

Did not find user configurations in /home/c-cc2016/ra173608/tramita/pyproject.toml.

## Funções, macros

In [2]:
import json
from pathlib import Path
import pyarrow as pa
import pyarrow.dataset as ds


def peek_json_payloads(
    root: str = "data/bronze/snapshots/bronze-2020-2024-v1",
    entity: str = "camara/proposicoes/details",
    *,
    n: int = 5,
    year: int | None = None,
) -> list[dict]:
    """
    Stream up to `n` JSON payloads from a hive-partitioned parquet dataset.
    Works across PyArrow versions (no scanner(limit=...) / to_table(limit=...)).
    """
    base = Path(root).as_posix().rstrip("/")
    path = f"{base}/{entity.strip('/')}"
    dataset = ds.dataset(path, format="parquet", partitioning="hive")

    names = set(dataset.schema.names)
    col = "payload_json" if "payload_json" in names else ("payload" if "payload" in names else None)
    if col is None:
        raise ValueError(f"No payload column found at {path}. Expected 'payload_json' or 'payload'.")

    flt = (ds.field("year") == year) if (year is not None and "year" in names) else None

    # Build a scanner without a limit and stream batches
    scanner = ds.Scanner.from_dataset(dataset, columns=[col], filter=flt)

    out: list[dict] = []
    for batch in scanner.to_batches():  # stream; no memory blow-up
        arr = batch.column(0)
        # slice if this batch would push us over n
        room = n - len(out)
        if room <= 0:
            break
        if len(arr) > room:
            arr = arr.slice(0, room)

        for v in arr.to_pylist():
            if v is None:
                continue
            if isinstance(v, (str, bytes, bytearray)):
                s = v.decode("utf-8") if isinstance(v, (bytes, bytearray)) else v
                try:
                    out.append(json.loads(s))
                except Exception:
                    out.append({"_raw": s})
            elif isinstance(v, dict):
                out.append(v)
            else:
                out.append({"_raw": v})

        if len(out) >= n:
            break

    # Pretty-print for quick inspection
    print(json.dumps(out, ensure_ascii=False, indent=2))
    return out


In [3]:
from __future__ import annotations

import json
from pathlib import Path
import pyarrow.dataset as ds


def peek_rows(
    root: str = "data/bronze/snapshots/bronze-2020-2024-v1",
    entity: str = "camara/proposicoes/details",
    *,
    n: int = 5,
    year: int | None = None,
) -> list[dict]:
    """
    Stream up to `n` full rows (all columns) from a hive-partitioned parquet dataset.
    Converts Arrow rows to dicts for easy inspection.
    """
    base = Path(root).as_posix().rstrip("/")
    path = f"{base}/{entity.strip('/')}"
    dataset = ds.dataset(path, format="parquet", partitioning="hive")

    flt = (ds.field("year") == year) if (year is not None and "year" in dataset.schema.names) else None

    scanner = ds.Scanner.from_dataset(dataset, filter=flt)

    out: list[dict] = []
    for batch in scanner.to_batches():
        room = n - len(out)
        if room <= 0:
            break

        tbl = batch.to_pydict()
        # batch.to_pydict() gives {col: list}, need row-wise dicts
        num_rows = min(len(next(iter(tbl.values()))), room)
        for i in range(num_rows):
            row = {k: v[i] for k, v in tbl.items()}
            # Try to parse payload_json nicely if present
            if "payload_json" in row and isinstance(row["payload_json"], str):
                try:
                    row["payload_json"] = json.loads(row["payload_json"])
                except Exception:
                    pass
            out.append(row)

        if len(out) >= n:
            break

    print(json.dumps(out, ensure_ascii=False, indent=2))
    return out


In [4]:
%%sql

CREATE OR REPLACE MACRO jget1(j, p) AS json_extract_string(j, p);
CREATE OR REPLACE MACRO jcoalesce(j, p1, p2, p3, p4, p5, p6) AS
  COALESCE(jget1(j, p1), jget1(j, p2), jget1(j, p3), jget1(j, p4), jget1(j, p5), jget1(j, p6));

Running query in 'duck'

Count


# Senado Federal

## Modelos dimensionais

### Bloco

In [5]:
%%sql
CREATE OR REPLACE VIEW bronze_senado_bloco AS
SELECT * FROM parquet_scan("data/bronze/snapshots/bronze-2020-2024-v1/senado/bloco/details/year=*/part-*.parquet")
WHERE source = 'senado' AND entity = 'bloco';

SELECT COUNT(*) AS n FROM bronze_senado_bloco;


Running query in 'duck'

n
6


In [6]:
%%sql
SELECT * FROM bronze_senado_bloco;

Running query in 'duck'

source,entity,id,url,payload_json,payload_sha256,year
senado,bloco,335,https://legis.senado.leg.br/dadosabertos/dados/ListaBlocoParlamentar.json,"{""CodigoBloco"":""335"",""DataCriacao"":""2023-02-01"",""Membros"":{""Membro"":[{""DataAdesao"":""2023-04-25"",""DataDesligamento"":""2024-01-30"",""Partido"":{""CodigoPartido"":""565"",""NomePartido"":""Rede Sustentabilidade"",""SiglaPartido"":""REDE""}},{""DataAdesao"":""2023-02-01"",""Partido"":{""CodigoPartido"":""63"",""NomePartido"":""Partido Socialista Brasileiro"",""SiglaPartido"":""PSB""}},{""DataAdesao"":""2023-02-01"",""DataDesligamento"":""2023-02-01"",""Partido"":{""CodigoPartido"":""66"",""NomePartido"":""Partido da Social Democracia Brasileira"",""SiglaPartido"":""PSDB""}},{""DataAdesao"":""2023-02-01"",""DataDesligamento"":""2025-02-17"",""Partido"":{""CodigoPartido"":""71"",""NomePartido"":""Partido dos Trabalhadores"",""SiglaPartido"":""PT""}},{""DataAdesao"":""2023-02-01"",""Partido"":{""CodigoPartido"":""557"",""NomePartido"":""Partido Social Democrático"",""SiglaPartido"":""PSD""}},{""DataAdesao"":""2023-02-01"",""DataDesligamento"":""2023-02-01"",""Partido"":{""CodigoPartido"":""565"",""NomePartido"":""Rede Sustentabilidade"",""SiglaPartido"":""REDE""}}]},""NomeApelido"":""BLPRD"",""NomeBloco"":""Bloco Parlamentar da Resistência Democrática""}",17bb2bcc628caccad8d592ed0ae509d8d6aff8e9d03f04324ca84857ad41349f,2023
senado,bloco,337,https://legis.senado.leg.br/dadosabertos/dados/ListaBlocoParlamentar.json,"{""CodigoBloco"":""337"",""DataCriacao"":""2023-02-03"",""NomeApelido"":""Minoria"",""NomeBloco"":""Minoria""}",e05b037fcdb0ba14965f881c4b889e3536a9830d1ecd9a4b4526f8fbc283b44f,2023
senado,bloco,346,https://legis.senado.leg.br/dadosabertos/dados/ListaBlocoParlamentar.json,"{""CodigoBloco"":""346"",""DataCriacao"":""2023-03-20"",""Membros"":{""Membro"":[{""DataAdesao"":""2023-03-20"",""Partido"":{""CodigoPartido"":""418"",""NomePartido"":""Progressistas"",""SiglaPartido"":""PP""}},{""DataAdesao"":""2023-03-20"",""Partido"":{""CodigoPartido"":""521"",""NomePartido"":""REPUBLICANOS"",""SiglaPartido"":""REPUBLICANOS""}}]},""NomeApelido"":""BLALIANÇA"",""NomeBloco"":""Bloco Parlamentar Aliança""}",0bfad6ac5099f84aef8d5d7b5cf7e2c7de9343b10e14bd5c9ad76171efba426a,2023
senado,bloco,357,https://legis.senado.leg.br/dadosabertos/dados/ListaBlocoParlamentar.json,"{""CodigoBloco"":""357"",""DataCriacao"":""2025-02-03"",""Membros"":{""Membro"":[{""DataAdesao"":""2025-02-03"",""Partido"":{""CodigoPartido"":""523"",""NomePartido"":""Partido Liberal"",""SiglaPartido"":""PL""}},{""DataAdesao"":""2025-02-03"",""Partido"":{""CodigoPartido"":""566"",""NomePartido"":""Partido Novo"",""SiglaPartido"":""NOVO""}}]},""NomeApelido"":""BLVANG"",""NomeBloco"":""Bloco Parlamentar Vanguarda""}",329482b8ae5ed25112eeece8b3425cde3cee1531352b728789a3fe6c16b8e09a,2025
senado,bloco,358,https://legis.senado.leg.br/dadosabertos/dados/ListaBlocoParlamentar.json,"{""CodigoBloco"":""358"",""DataCriacao"":""2025-02-18"",""Membros"":{""Membro"":[{""DataAdesao"":""2025-02-18"",""Partido"":{""CodigoPartido"":""582"",""NomePartido"":""União Brasil"",""SiglaPartido"":""UNIÃO""}},{""DataAdesao"":""2025-02-18"",""Partido"":{""CodigoPartido"":""577"",""NomePartido"":""Podemos"",""SiglaPartido"":""PODEMOS""}},{""DataAdesao"":""2025-02-18"",""Partido"":{""CodigoPartido"":""33"",""NomePartido"":""Movimento Democrático Brasileiro"",""SiglaPartido"":""MDB""}},{""DataAdesao"":""2025-02-18"",""Partido"":{""CodigoPartido"":""66"",""NomePartido"":""Partido da Social Democracia Brasileira"",""SiglaPartido"":""PSDB""}}]},""NomeApelido"":""BLDEM"",""NomeBloco"":""Bloco Parlamentar Democracia""}",7a8f88b1a99dcd2e88fdeb67839b2ceb62ce1883ac778de28c5960c8d0890930,2025
senado,bloco,359,https://legis.senado.leg.br/dadosabertos/dados/ListaBlocoParlamentar.json,"{""CodigoBloco"":""359"",""DataCriacao"":""2025-02-18"",""Membros"":{""Membro"":[{""DataAdesao"":""2025-02-18"",""Partido"":{""CodigoPartido"":""20"",""NomePartido"":""Part

In [7]:

%%sql
DROP TABLE IF EXISTS bloco_senado;
CREATE TABLE bloco_senado AS
WITH base AS (
    SELECT TRY_CAST(payload_json AS JSON) as j, year
    FROM bronze_senado_bloco
)
SELECT
    -- CodigoBloco
    CAST(jget1(j, '$.CodigoBloco') AS BIGINT) AS codigo_bloco,
    -- DataCriacao
    CAST(jget1(j, '$.DataCriacao') AS DATE) AS data_criacao,
    -- NomeApelido
    jget1(j, '$.NomeApelido') AS nome_apelido,
    -- NomeBloco
    jget1(j, '$.NomeBloco') AS nome_bloco,
    year AS year_snapshot
FROM base
WHERE jget1(j, '$.CodigoBloco') IS NOT NULL;
DROP VIEW IF EXISTS bronze_senado_bloco;


Running query in 'duck'

Success


In [8]:
%%sql
SELECT * FROM bloco_senado LIMIT 5;

Running query in 'duck'

codigo_bloco,data_criacao,nome_apelido,nome_bloco,year_snapshot
335,2023-02-01,BLPRD,Bloco Parlamentar da Resistência Democrática,2023
337,2023-02-03,Minoria,Minoria,2023
346,2023-03-20,BLALIANÇA,Bloco Parlamentar Aliança,2023
357,2025-02-03,BLVANG,Bloco Parlamentar Vanguarda,2025
358,2025-02-18,BLDEM,Bloco Parlamentar Democracia,2025


### Colegiado

In [9]:
%%sql
CREATE OR REPLACE VIEW bronze_senado_colegiado AS
SELECT * FROM parquet_scan("data/bronze/snapshots/bronze-2020-2024-v1/senado/colegiado/details/year=*/part-*.parquet")
WHERE source = 'senado' AND entity = 'colegiado';

SELECT COUNT(*) AS n FROM bronze_senado_colegiado;

Running query in 'duck'

n
18


In [10]:
%%sql
SELECT * FROM bronze_senado_colegiado;

Running query in 'duck'

source,entity,id,url,payload_json,payload_sha256,year
senado,colegiado,34,https://legis.senado.leg.br/dadosabertos/comissao/lista/permanente.json,"{""CodigoColegiado"":""34"",""CodigoTipoColegiado"":""21"",""DataInicio"":""1900-01-01"",""IndicadorDistrPartidaria"":""S"",""NomeColegiado"":""Comissão de Constituição, Justiça e Cidadania"",""SiglaColegiado"":""CCJ"",""id"":""34"",""ordem"":""3"",""tipocolegiado"":{""CodigoNaturezaColegiado"":""1"",""CodigoTipoColegiado"":""21"",""DescricaoTipoColegiado"":""Comissão Permanente"",""IndicadorAtivo"":""S"",""SiglaCasa"":""SF"",""SiglaTipoColegiado"":""PERMANENTE""}}",78ed658e3085b205f0f04ba4a60012b4ffb82a4c940b9002175b607892541f8c,1900
senado,colegiado,38,https://legis.senado.leg.br/dadosabertos/comissao/lista/permanente.json,"{""CodigoColegiado"":""38"",""CodigoTipoColegiado"":""21"",""DataInicio"":""1900-01-01"",""IndicadorDistrPartidaria"":""S"",""NomeColegiado"":""Comissão de Assuntos Econômicos"",""SiglaColegiado"":""CAE"",""id"":""38"",""ordem"":""1"",""tipocolegiado"":{""CodigoNaturezaColegiado"":""1"",""CodigoTipoColegiado"":""21"",""DescricaoTipoColegiado"":""Comissão Permanente"",""IndicadorAtivo"":""S"",""SiglaCasa"":""SF"",""SiglaTipoColegiado"":""PERMANENTE""}}",5b3ab8402aca0640913710c9d437529da92d924e0e88933cbdbcd76ad8c1f8e4,1900
senado,colegiado,40,https://legis.senado.leg.br/dadosabertos/comissao/lista/permanente.json,"{""CodigoColegiado"":""40"",""CodigoTipoColegiado"":""21"",""DataInicio"":""1900-01-01"",""IndicadorDistrPartidaria"":""S"",""NomeColegiado"":""Comissão de Assuntos Sociais"",""SiglaColegiado"":""CAS"",""id"":""40"",""ordem"":""2"",""tipocolegiado"":{""CodigoNaturezaColegiado"":""1"",""CodigoTipoColegiado"":""21"",""DescricaoTipoColegiado"":""Comissão Permanente"",""IndicadorAtivo"":""S"",""SiglaCasa"":""SF"",""SiglaTipoColegiado"":""PERMANENTE""}}",35d4234190deb511703364728fb000edb326e0659b064c26d75149a5cba8fa0a,1900
senado,colegiado,47,https://legis.senado.leg.br/dadosabertos/comissao/lista/permanente.json,"{""CodigoColegiado"":""47"",""CodigoTipoColegiado"":""21"",""DataInicio"":""1900-01-01"",""IndicadorDistrPartidaria"":""S"",""NomeColegiado"":""Comissão de Educação e Cultura"",""SiglaColegiado"":""CE"",""id"":""47"",""ordem"":""4"",""tipocolegiado"":{""CodigoNaturezaColegiado"":""1"",""CodigoTipoColegiado"":""21"",""DescricaoTipoColegiado"":""Comissão Permanente"",""IndicadorAtivo"":""S"",""SiglaCasa"":""SF"",""SiglaTipoColegiado"":""PERMANENTE""}}",6f59be0b517199c781da41f2612350d986a34843c900d0d58cb10306a54b86bd,1900
senado,colegiado,50,https://legis.senado.leg.br/dadosabertos/comissao/lista/permanente.json,"{""CodigoColegiado"":""50"",""CodigoTipoColegiado"":""21"",""DataInicio"":""1900-01-01"",""IndicadorDistrPartidaria"":""S"",""NomeColegiado"":""Comissão de Meio Ambiente"",""SiglaColegiado"":""CMA"",""id"":""50"",""ordem"":""13"",""tipocolegiado"":{""CodigoNaturezaColegiado"":""1"",""CodigoTipoColegiado"":""21"",""DescricaoTipoColegiado"":""Comissão Permanente"",""IndicadorAtivo"":""S"",""SiglaCasa"":""SF"",""SiglaTipoColegiado"":""PERMANENTE""}}",a3212efba1e5cb3bcffde359a325abf86675cd10ddbe710974f910a29634a39e,1900
senado,colegiado,54,https://legis.senado.leg.br/dadosabertos/comissao/lista/permanente.json,"{""CodigoColegiado"":""54"",""CodigoTipoColegiado"":""21"",""DataInicio"":""1900-01-01"",""IndicadorDistrPartidaria"":""S"",""NomeColegiado"":""Comissão de Relações Exteriores e Defesa Nacional"",""SiglaColegiado"":""CRE"",""id"":""54"",""ordem"":""7"",""tipocolegiado"":{""CodigoNaturezaColegiado"":""1"",""CodigoTipoColegiado"":""21"",""DescricaoTipoColegiado"":""Comissão Permanente"",""IndicadorAtivo"":""S"",""SiglaCasa"":""SF"",""SiglaTipoColegiado"":""PERMANENTE""}}",bc7cf00419f2cf7357358e61081495ee226d6f8c745c0b1306333a16d2d0ba19,1900
senado,colegiado,59,https://legis.senado.leg.br/dadosabertos/comissao/lista/permanente.json,"{""CodigoColegiado"":""59"",""CodigoTipoColegiado"":""21"",""DataInicio"":""1900-01-01"",""IndicadorDistrPartid

In [11]:
peek_json_payloads(entity="senado/colegiado/details")

[
  {
    "CodigoColegiado": "34",
    "CodigoTipoColegiado": "21",
    "DataInicio": "1900-01-01",
    "IndicadorDistrPartidaria": "S",
    "NomeColegiado": "Comissão de Constituição, Justiça e Cidadania",
    "SiglaColegiado": "CCJ",
    "id": "34",
    "ordem": "3",
    "tipocolegiado": {
      "CodigoNaturezaColegiado": "1",
      "CodigoTipoColegiado": "21",
      "DescricaoTipoColegiado": "Comissão Permanente",
      "IndicadorAtivo": "S",
      "SiglaCasa": "SF",
      "SiglaTipoColegiado": "PERMANENTE"
    }
  },
  {
    "CodigoColegiado": "38",
    "CodigoTipoColegiado": "21",
    "DataInicio": "1900-01-01",
    "IndicadorDistrPartidaria": "S",
    "NomeColegiado": "Comissão de Assuntos Econômicos",
    "SiglaColegiado": "CAE",
    "id": "38",
    "ordem": "1",
    "tipocolegiado": {
      "CodigoNaturezaColegiado": "1",
      "CodigoTipoColegiado": "21",
      "DescricaoTipoColegiado": "Comissão Permanente",
      "IndicadorAtivo": "S",
      "SiglaCasa": "SF",
      "SiglaTi

[{'CodigoColegiado': '34',
  'CodigoTipoColegiado': '21',
  'DataInicio': '1900-01-01',
  'IndicadorDistrPartidaria': 'S',
  'NomeColegiado': 'Comissão de Constituição, Justiça e Cidadania',
  'SiglaColegiado': 'CCJ',
  'id': '34',
  'ordem': '3',
  'tipocolegiado': {'CodigoNaturezaColegiado': '1',
   'CodigoTipoColegiado': '21',
   'DescricaoTipoColegiado': 'Comissão Permanente',
   'IndicadorAtivo': 'S',
   'SiglaCasa': 'SF',
   'SiglaTipoColegiado': 'PERMANENTE'}},
 {'CodigoColegiado': '38',
  'CodigoTipoColegiado': '21',
  'DataInicio': '1900-01-01',
  'IndicadorDistrPartidaria': 'S',
  'NomeColegiado': 'Comissão de Assuntos Econômicos',
  'SiglaColegiado': 'CAE',
  'id': '38',
  'ordem': '1',
  'tipocolegiado': {'CodigoNaturezaColegiado': '1',
   'CodigoTipoColegiado': '21',
   'DescricaoTipoColegiado': 'Comissão Permanente',
   'IndicadorAtivo': 'S',
   'SiglaCasa': 'SF',
   'SiglaTipoColegiado': 'PERMANENTE'}},
 {'CodigoColegiado': '40',
  'CodigoTipoColegiado': '21',
  'DataIni

In [12]:
%%sql
DROP TABLE IF EXISTS colegiado_senado;
CREATE TABLE colegiado_senado AS
WITH base AS (
    SELECT TRY_CAST(payload_json AS JSON) as j, year
    FROM bronze_senado_colegiado
)
SELECT
    -- CodigoColegiado
    CAST(jget1(j, '$.CodigoColegiado') AS BIGINT) AS codigo_colegiado,
    -- CodigoTipoColegiado
    CAST(jget1(j, '$.CodigoTipoColegiado') AS BIGINT) AS codigo_tipo_colegiado,
    -- DataInicio
    CAST(jget1(j, '$.DataInicio') AS DATE) AS data_inicio,
    -- IndicadorDistrPartidaria
    jget1(j, '$.IndicadorDistrPartidaria') AS indicador_distr_partidaria,
    -- NomeColegiado
    jget1(j, '$.NomeColegiado') AS nome_colegiado,
    -- SiglaColegiado
    jget1(j, '$.SiglaColegiado') AS sigla_colegiado,
    -- ordem
    CAST(jget1(j, '$.ordem') AS INTEGER) AS ordem,
    year AS year_snapshot
FROM base
WHERE jget1(j, '$.CodigoColegiado') IS NOT NULL;

DROP TABLE IF EXISTS tipo_colegiado_senado;
CREATE TABLE tipo_colegiado_senado AS
WITH base AS (
    SELECT TRY_CAST(payload_json AS JSON) AS j, year
    FROM bronze_senado_colegiado
),
typed AS (
    SELECT
        CAST(jget1(j, '$.tipocolegiado.CodigoNaturezaColegiado') AS BIGINT) AS codigo_natureza_colegiado,
        CAST(jget1(j, '$.tipocolegiado.CodigoTipoColegiado') AS BIGINT) AS codigo_tipo_colegiado,
        jget1(j, '$.tipocolegiado.DescricaoTipoColegiado') AS descricao_tipo_colegiado,
        jget1(j, '$.tipocolegiado.IndicadorAtivo') AS indicador_ativo,
        jget1(j, '$.tipocolegiado.SiglaCasa') AS sigla_casa,
        jget1(j, '$.tipocolegiado.SiglaTipoColegiado') AS sigla_tipo_colegiado,
        year AS year_snapshot
    FROM base
    WHERE jget1(j, '$.tipocolegiado.CodigoTipoColegiado') IS NOT NULL
)
SELECT
    codigo_tipo_colegiado,
    arg_max(codigo_natureza_colegiado, year_snapshot) AS codigo_natureza_colegiado,
    arg_max(descricao_tipo_colegiado,  year_snapshot) AS descricao_tipo_colegiado,
    arg_max(indicador_ativo,           year_snapshot) AS indicador_ativo,
    arg_max(sigla_casa,                year_snapshot) AS sigla_casa,
    arg_max(sigla_tipo_colegiado,      year_snapshot) AS sigla_tipo_colegiado,
    max(year_snapshot)                              AS year_snapshot
FROM typed
GROUP BY codigo_tipo_colegiado;



DROP VIEW IF EXISTS bronze_senado_bloco;

Running query in 'duck'

Success


In [13]:
%%sql
SELECT * FROM colegiado_senado;

Running query in 'duck'

codigo_colegiado,codigo_tipo_colegiado,data_inicio,indicador_distr_partidaria,nome_colegiado,sigla_colegiado,ordem,year_snapshot
34,21,1900-01-01,S,"Comissão de Constituição, Justiça e Cidadania",CCJ,3,1900
38,21,1900-01-01,S,Comissão de Assuntos Econômicos,CAE,1,1900
40,21,1900-01-01,S,Comissão de Assuntos Sociais,CAS,2,1900
47,21,1900-01-01,S,Comissão de Educação e Cultura,CE,4,1900
50,21,1900-01-01,S,Comissão de Meio Ambiente,CMA,13,1900
54,21,1900-01-01,S,Comissão de Relações Exteriores e Defesa Nacional,CRE,7,1900
59,21,1900-01-01,S,Comissão de Serviços de Infraestrutura,CI,8,1900
834,21,2002-12-13,S,Comissão de Direitos Humanos e Legislação Participativa,CDH,6,2002
1306,21,2005-02-18,S,Comissão de Desenvolvimento Regional e Turismo,CDR,9,2005
1307,21,2005-02-22,S,Comissão de Agricultura e Reforma Agrária,CRA,10,2005


In [14]:
%%sql
SELECT * FROM tipo_colegiado_senado;

Running query in 'duck'

codigo_tipo_colegiado,codigo_natureza_colegiado,descricao_tipo_colegiado,indicador_ativo,sigla_casa,sigla_tipo_colegiado,year_snapshot
21,1,Comissão Permanente,S,SF,PERMANENTE,2023
121,2,Comissão Temporária Externa,S,SF,CT,2025


### Parlamentar

In [15]:
%%sql
CREATE OR REPLACE VIEW bronze_senado_parlamentar AS
SELECT * FROM parquet_scan("data/bronze/snapshots/bronze-2020-2024-v1/senado/parlamentar/details/year=*/part-*.parquet")
WHERE source = 'senado' AND entity = 'parlamentar';

SELECT COUNT(*) AS n FROM bronze_senado_parlamentar;

Running query in 'duck'

n
1298


In [16]:
%%sql
SELECT * FROM bronze_senado_parlamentar LIMIT 1;

Running query in 'duck'

source,entity,id,url,payload_json,payload_sha256,year
senado,parlamentar,22,https://legis.senado.leg.br/dadosabertos/senador/22.json,"{""DetalheParlamentar"":{""Metadados"":{""DataVersaoServico"":""2021-09-09"",""DescricaoDataSet"":""Retorna os detalhes de um Senador informado no parâmetro."",""Versao"":""22/09/2025 20:06:34"",""VersaoServico"":""6""},""Parlamentar"":{""DadosBasicosParlamentar"":{""DataNascimento"":""1947-12-21"",""EnderecoParlamentar"":""Senado Federal Anexo 2 Pavimento Térreo ALA NILO COELHO Gabinete 02"",""Naturalidade"":""Florianópolis"",""UfNaturalidade"":""SC""},""IdentificacaoParlamentar"":{""CodigoParlamentar"":""22"",""CodigoPublicoNaLegAtual"":""882"",""EmailParlamentar"":""sen.esperidiaoamin@senado.leg.br"",""NomeCompletoParlamentar"":""Esperidião Amin Helou Filho"",""NomeParlamentar"":""Esperidião Amin"",""SexoParlamentar"":""Masculino"",""SiglaPartidoParlamentar"":""PP"",""UfParlamentar"":""SC"",""UrlFotoParlamentar"":""http://www.senado.leg.br/senadores/img/fotos-oficiais/senador22.jpg"",""UrlPaginaParlamentar"":""http://www25.senado.leg.br/web/senadores/senador/-/perfil/22""},""OutrasInformacoes"":{""Servico"":[{""DescricaoServico"":""Apartes do parlamentar"",""NomeServico"":""ApartesParlamentar"",""UrlServico"":""https://legis.senado.leg.br/dadosabertos/senador/22/apartes?v=5""},{""DescricaoServico"":""Cargos do parlamentar ao longo do(s) mandato(s)."",""NomeServico"":""CargoParlamentar"",""UrlServico"":""https://legis.senado.leg.br/dadosabertos/senador/22/cargos?v=5""},{""DescricaoServico"":""Discursos do parlamentar. Se não informar o período, serão retornados os discursos dos últimos 30 dias."",""NomeServico"":""DiscursosParlamentar"",""UrlServico"":""https://legis.senado.leg.br/dadosabertos/senador/22/discursos?v=5""},{""NomeServico"":""FiliacaoParlamentar"",""UrlServico"":""https://legis.senado.leg.br/dadosabertos/senador/22/filiacoes?v=5""},{""DescricaoServico"":""Retorna o histporico acadêmico do parlamentar."",""NomeServico"":""HistoricoAcademicoParlamentar"",""UrlServico"":""https://legis.senado.leg.br/dadosabertos/senador/historicoAcademico/22/historicoAcademico?v=1""},{""DescricaoServico"":""Licenças do parlamentar."",""NomeServico"":""LicencaParlamentar"",""UrlServico"":""https://legis.senado.leg.br/dadosabertos/senador/22/licencas?v=6""},{""DescricaoServico"":""Cargos de liderança do parlamentar ao longo do(s) mandato(s)"",""NomeServico"":""LiderancaParlamentar"",""UrlServico"":""https://legis.senado.leg.br/dadosabertos/senador/22/liderancas?v=5""},{""DescricaoServico"":""Mandatos do parlamentar (atual e anteriores, se houver)"",""NomeServico"":""MandatoParlamentar"",""UrlServico"":""https://legis.senado.leg.br/dadosabertos/senador/22/mandatos?v=5""},{""DescricaoServico"":""Matérias de autoria ou coautoria do parlamentar"",""NomeServico"":""MateriasAutoriaParlamentar"",""UrlServico"":""https://legis.senado.leg.br/dadosabertos/senador/22/autorias?v=7""},{""DescricaoServico"":""Matérias de relatoria do parlamentar"",""NomeServico"":""MateriasRelatoriaParlamentar"",""UrlServico"":""https://legis.senado.leg.br/dadosabertos/senador/22/relatorias?v=6""},{""DescricaoServico"":""Comissões em que o parlamentar é (ou foi) membro"",""NomeServico"":""MembroComissaoParlamentar"",""UrlServico"":""https://legis.senado.leg.br/dadosabertos/senador/22/comissoes?v=5""},{""DescricaoServico"":""Retorna a(s) profissão(ões) do parlamentar."",""NomeServico"":""ProfissaoParlamentar"",""UrlServico"":""https://legis.senado.leg.br/dadosabertos/senador/profissao/22/profissoes?v=1""},{""DescricaoServico"":""Votos do parlamentar ao longo do(s) mandato(s)"",""NomeServico"":""VotacaoParlamentar"",""UrlServico"":""https://legis.senado.leg.br/dadosabertos/senador/22/votacoes?v=7""}]},""Telefones"":{""Telefone"":[{""IndicadorFax"":""Não"",""NumeroTelefone"":""33036446"",""OrdemPublicacao"":""1""},{""IndicadorFax"":""Não"",""NumeroTelefone"":""33036447"",""OrdemPublicacao"":""2""},{""IndicadorFax"":""Não"",""NumeroTel

In [17]:
peek_json_payloads(entity='senado/parlamentar/details')

[
  {
    "DetalheParlamentar": {
      "Metadados": {
        "DataVersaoServico": "2021-09-09",
        "DescricaoDataSet": "Retorna os detalhes de um Senador informado no parâmetro.",
        "Versao": "22/09/2025 20:06:34",
        "VersaoServico": "6"
      },
      "Parlamentar": {
        "DadosBasicosParlamentar": {
          "DataNascimento": "1947-12-21",
          "EnderecoParlamentar": "Senado Federal Anexo 2  Pavimento Térreo ALA NILO COELHO Gabinete 02",
          "Naturalidade": "Florianópolis",
          "UfNaturalidade": "SC"
        },
        "IdentificacaoParlamentar": {
          "CodigoParlamentar": "22",
          "CodigoPublicoNaLegAtual": "882",
          "EmailParlamentar": "sen.esperidiaoamin@senado.leg.br",
          "NomeCompletoParlamentar": "Esperidião Amin Helou Filho",
          "NomeParlamentar": "Esperidião Amin",
          "SexoParlamentar": "Masculino",
          "SiglaPartidoParlamentar": "PP",
          "UfParlamentar": "SC",
          "UrlFotoPar

[{'DetalheParlamentar': {'Metadados': {'DataVersaoServico': '2021-09-09',
    'DescricaoDataSet': 'Retorna os detalhes de um Senador informado no parâmetro.',
    'Versao': '22/09/2025 20:06:34',
    'VersaoServico': '6'},
   'Parlamentar': {'DadosBasicosParlamentar': {'DataNascimento': '1947-12-21',
     'EnderecoParlamentar': 'Senado Federal Anexo 2  Pavimento Térreo ALA NILO COELHO Gabinete 02',
     'Naturalidade': 'Florianópolis',
     'UfNaturalidade': 'SC'},
    'IdentificacaoParlamentar': {'CodigoParlamentar': '22',
     'CodigoPublicoNaLegAtual': '882',
     'EmailParlamentar': 'sen.esperidiaoamin@senado.leg.br',
     'NomeCompletoParlamentar': 'Esperidião Amin Helou Filho',
     'NomeParlamentar': 'Esperidião Amin',
     'SexoParlamentar': 'Masculino',
     'SiglaPartidoParlamentar': 'PP',
     'UfParlamentar': 'SC',
     'UrlFotoParlamentar': 'http://www.senado.leg.br/senadores/img/fotos-oficiais/senador22.jpg',
     'UrlPaginaParlamentar': 'http://www25.senado.leg.br/web/se

In [18]:
%%sql
DROP TABLE IF EXISTS parlamentar_senado;

CREATE TABLE parlamentar_senado AS
WITH base AS (
    SELECT TRY_CAST(payload_json AS JSON) AS j, year
    FROM bronze_senado_parlamentar
),
par_root AS (
    SELECT
        json_extract(j, '$.DetalheParlamentar.Parlamentar') AS par,
        year
    FROM base
),
flattened AS (
    SELECT
        -- Identificação
        CAST(jget1(par, '$.IdentificacaoParlamentar.CodigoParlamentar') AS BIGINT) AS codigo_parlamentar,
        CAST(jget1(par, '$.IdentificacaoParlamentar.CodigoPublicoNaLegAtual') AS BIGINT) AS codigo_publico_leg_atual,
        jget1(par, '$.IdentificacaoParlamentar.NomeCompletoParlamentar') AS nome_completo,
        jget1(par, '$.IdentificacaoParlamentar.NomeParlamentar') AS nome_parlamentar,
        jget1(par, '$.IdentificacaoParlamentar.SexoParlamentar') AS sexo_parlamentar,
        jget1(par, '$.IdentificacaoParlamentar.SiglaPartidoParlamentar') AS sigla_partido,
        jget1(par, '$.IdentificacaoParlamentar.UfParlamentar') AS uf_parlamentar,
        jget1(par, '$.IdentificacaoParlamentar.EmailParlamentar') AS email_parlamentar,
        -- Dados básicos
        CAST(jget1(par, '$.DadosBasicosParlamentar.DataNascimento') AS DATE) AS data_nascimento,
        jget1(par, '$.DadosBasicosParlamentar.EnderecoParlamentar') AS endereco_parlamentar,
        jget1(par, '$.DadosBasicosParlamentar.Naturalidade') AS naturalidade,
        jget1(par, '$.DadosBasicosParlamentar.UfNaturalidade') AS uf_naturalidade,
        year AS year_snapshot
    FROM par_root
)
SELECT *
FROM flattened
WHERE codigo_parlamentar IS NOT NULL;

DROP VIEW IF EXISTS bronze_senado_parlamentar;


Running query in 'duck'

Success


In [19]:
%%sql
SELECT * FROM parlamentar_senado LIMIT 5;

Running query in 'duck'

codigo_parlamentar,codigo_publico_leg_atual,nome_completo,nome_parlamentar,sexo_parlamentar,sigla_partido,uf_parlamentar,email_parlamentar,data_nascimento,endereco_parlamentar,naturalidade,uf_naturalidade,year_snapshot
22,882,Esperidião Amin Helou Filho,Esperidião Amin,Masculino,PP,SC,sen.esperidiaoamin@senado.leg.br,1947-12-21,Senado Federal Anexo 2 Pavimento Térreo ALA NILO COELHO Gabinete 02,Florianópolis,SC,2020
35,932,Jader Fontenelle Barbalho,Jader Barbalho,Masculino,MDB,PA,sen.jaderbarbalho@senado.leg.br,1944-10-27,Senado Federal Anexo 1 2º Pavimento,Belém,PA,2020
63,948,Ney Robinson Suassuna,Ney Suassuna,Masculino,REPUBLICANOS,None,sen.neysuassuna@senado.leg.br,1941-10-11,Senado Federal Anexo 1 20º Pavimento,Campina Grande,PB,2020
70,875,José Renan Vasconcelos Calheiros,Renan Calheiros,Masculino,MDB,AL,sen.renancalheiros@senado.leg.br,1955-09-16,Senado Federal Anexo 1 15º Pavimento,Murici,AL,2020
90,846,José Serra,José Serra,Masculino,PSDB,None,sen.joseserra@senado.leg.br,1942-03-19,Senado Federal Edifício Principal Ala Dinarte Mariz Gabinete 02,São Paulo,SP,2020


### Partido

In [20]:
%%sql
CREATE OR REPLACE VIEW bronze_senado_partido AS
SELECT * FROM parquet_scan("data/bronze/snapshots/bronze-2020-2024-v1/senado/partido/details/year=*/part-*.parquet")
WHERE source = 'senado' AND entity = 'partido';

SELECT COUNT(*) AS n FROM bronze_senado_partido;

Running query in 'duck'

n
103


In [21]:
%%sql
SELECT * FROM bronze_senado_partido LIMIT 1;

Running query in 'duck'

source,entity,id,url,payload_json,payload_sha256,year
senado,partido,146,https://legis.senado.leg.br/dadosabertos/dados/ListaPartidos.json,"{""Codigo"":""146"",""DataCriacao"":""1900-01-01"",""Nome"":""Partido Federalista"",""Sigla"":""PF""}",acd8283711921d68649fc6a10fccb45c7596bbdc091d611d1c6a95c152544e8f,1900


In [22]:

%%sql
DROP TABLE IF EXISTS partido_senado;
CREATE TABLE partido_senado AS
WITH base AS (
    SELECT TRY_CAST(payload_json AS JSON) as j, year
    FROM bronze_senado_partido
)
SELECT
    CAST(jget1(j, '$.Codigo') AS BIGINT) AS codigo_partido,
    CAST(jget1(j, '$.DataCriacao') AS DATE) AS data_criacao,
    jget1(j, '$.Nome') AS nome,
    jget1(j, '$.Sigla') AS sigla,
    year AS year_snapshot
FROM base
WHERE jget1(j, '$.Codigo') IS NOT NULL;
DROP VIEW IF EXISTS bronze_senado_partido;

Running query in 'duck'

Success


In [23]:
%%sql
SELECT * FROM partido_senado LIMIT 5;

Running query in 'duck'

codigo_partido,data_criacao,nome,sigla,year_snapshot
146,1900-01-01,Partido Federalista,PF,1900
148,1900-01-01,Partido Humanista Democrático Brasil Solidariedade,PHDBS,1900
150,1900-01-01,Partido Nacional do Consumidor,PNC,1900
155,1900-01-01,Partido de Representação da Vontade Popular,PRVP,1900
163,1900-01-01,Sem Partido,S/Partido,1900


### Processo

In [24]:
%%sql
CREATE OR REPLACE VIEW bronze_senado_processo AS
SELECT * FROM parquet_scan("data/bronze/snapshots/bronze-2020-2024-v1/senado/processo/details/year=*/part-*.parquet")
WHERE source = 'senado' AND entity = 'processo';

SELECT COUNT(*) AS n FROM bronze_senado_processo;

Running query in 'duck'

n
19809


In [25]:
%%sql
SELECT * FROM bronze_senado_processo LIMIT 1;

Running query in 'duck'

+--------+----------+---------+---------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [26]:
peek_json_payloads(entity='senado/processo/details')

[
  {
    "ano": 2001,
    "apelido": "Multa relativa a informações sobre capitais brasileiros no exterior",
    "autoriaIniciativa": [
      {
        "autor": "Presidência da República",
        "descricaoTipo": "PRESIDENTE_REPUBLICA",
        "ente": "Presidência da República",
        "idEnte": 55126,
        "ordem": 1,
        "outrosAutoresNaoInformados": "Não",
        "siglaEnte": "PR",
        "siglaTipo": "PRESIDENTE_REPUBLICA"
      }
    ],
    "autuacoes": [
      {
        "descricao": "Autuação Principal",
        "idEnteControleAtual": 55304,
        "informesLegislativos": [
          {
            "colegiado": {},
            "data": "2001-09-06",
            "descricao": "Este processo contém 02 (duas) folhas numeradas e rubricadas.\nÀ SSCLCN.",
            "enteAdministrativo": {
              "casa": "SF",
              "id": 7217797,
              "nome": "PROTOCOLO LEGISLATIVO",
              "sigla": "PLEG"
            },
            "id": 863903,
            "

[{'ano': 2001,
  'apelido': 'Multa relativa a informações sobre capitais brasileiros no exterior',
  'autoriaIniciativa': [{'autor': 'Presidência da República',
    'descricaoTipo': 'PRESIDENTE_REPUBLICA',
    'ente': 'Presidência da República',
    'idEnte': 55126,
    'ordem': 1,
    'outrosAutoresNaoInformados': 'Não',
    'siglaEnte': 'PR',
    'siglaTipo': 'PRESIDENTE_REPUBLICA'}],
  'autuacoes': [{'descricao': 'Autuação Principal',
    'idEnteControleAtual': 55304,
    'informesLegislativos': [{'colegiado': {},
      'data': '2001-09-06',
      'descricao': 'Este processo contém 02 (duas) folhas numeradas e rubricadas.\nÀ SSCLCN.',
      'enteAdministrativo': {'casa': 'SF',
       'id': 7217797,
       'nome': 'PROTOCOLO LEGISLATIVO',
       'sigla': 'PLEG'},
      'id': 863903,
      'idSituacaoIniciada': 13,
      'siglaSituacaoIniciada': 'AGLEIT'},
     {'colegiado': {},
      'data': '2001-09-10',
      'descricao': '\n    Ao Plenário para designação da Comissão Mista e estab

In [27]:
%%sql
DROP TABLE IF EXISTS processo_senado;

CREATE TABLE processo_senado AS
WITH base AS (
    SELECT TRY_CAST(payload_json AS JSON) AS j, year
    FROM bronze_senado_processo
),
flat AS (
    SELECT
        -- identificadores principais
        CAST(jget1(j, '$.id') AS BIGINT)                       AS id_processo,
        CAST(jget1(j, '$.codigoMateria') AS BIGINT)            AS codigo_materia,
        CAST(jget1(j, '$.idProcessoCasaInicial') AS BIGINT)    AS id_processo_casa_inicial,

        jget1(j, '$.identificacao')                            AS identificacao,
        jget1(j, '$.identificacaoProcessoInicial')             AS identificacao_processo_inicial,
        jget1(j, '$.identificacaoExterna')                     AS identificacao_externa,

        -- campos simples de nível superior
        CAST(jget1(j, '$.ano') AS INTEGER)                     AS ano,
        jget1(j, '$.casaIdentificadora')                       AS casa_identificadora,
        jget1(j, '$.siglaCasaIniciadora')                      AS sigla_casa_iniciadora,
        jget1(j, '$.siglaEnteIdentificador')                   AS sigla_ente_identificador,
        jget1(j, '$.descricaoSigla')                           AS descricao_sigla,
        jget1(j, '$.sigla')                                    AS sigla,
        jget1(j, '$.numero')                                   AS numero,
        jget1(j, '$.objetivo')                                 AS objetivo,
        jget1(j, '$.tramitando')                               AS tramitando,

        -- conteúdo principal
        CAST(jget1(j, '$.conteudo.id') AS BIGINT)              AS id_conteudo,
        CAST(jget1(j, '$.conteudo.idTipo') AS BIGINT)          AS id_tipo_conteudo,
        jget1(j, '$.conteudo.siglaTipo')                       AS sigla_tipo_conteudo,
        jget1(j, '$.conteudo.tipo')                            AS tipo_conteudo,
        jget1(j, '$.conteudo.tipoNormaIndicada')               AS tipo_norma_indicada,
        jget1(j, '$.conteudo.ementa')                          AS ementa,
        jget1(j, '$.conteudo.explicacaoEmenta')                AS explicacao_ementa,

        -- deliberação final
        CAST(jget1(j, '$.deliberacao.idDestino') AS BIGINT)    AS deliberacao_id_destino,
        jget1(j, '$.deliberacao.siglaDestino')                 AS deliberacao_sigla_destino,
        jget1(j, '$.deliberacao.tipoDeliberacao')              AS deliberacao_tipo,
        jget1(j, '$.deliberacao.siglaTipo')                    AS deliberacao_sigla_tipo,
        CAST(jget1(j, '$.deliberacao.data') AS DATE)           AS deliberacao_data,
        jget1(j, '$.deliberacao.destino')                      AS deliberacao_destino,

        -- documento associado
        CAST(jget1(j, '$.documento.id') AS BIGINT)             AS id_documento,
        jget1(j, '$.documento.siglaTipo')                      AS documento_sigla_tipo,
        jget1(j, '$.documento.tipo')                           AS documento_tipo,
        jget1(j, '$.documento.indexacao')                      AS documento_indexacao,
        jget1(j, '$.documento.resumoAutoria')                  AS documento_resumo_autoria,
        CAST(jget1(j, '$.documento.dataApresentacao') AS DATE) AS documento_data_apresentacao,
        CAST(jget1(j, '$.documento.dataLeitura') AS DATE)      AS documento_data_leitura,

        -- norma gerada
        CAST(jget1(j, '$.normaGerada.codigo') AS BIGINT)       AS norma_codigo,
        jget1(j, '$.normaGerada.numero')                       AS norma_numero,
        jget1(j, '$.normaGerada.siglaTipo')                    AS norma_sigla_tipo,
        jget1(j, '$.normaGerada.tipo')                         AS norma_tipo,
        jget1(j, '$.normaGerada.descricao')                    AS norma_descricao,
        jget1(j, '$.normaGerada.siglaVeiculoPublicacao')       AS norma_sigla_veiculo,
        jget1(j, '$.normaGerada.veiculoPublicacao')            AS norma_veiculo,
        CAST(jget1(j, '$.normaGerada.numero') AS INTEGER)      AS norma_numero_int,
        CAST(jget1(j, '$.normaGerada.anoAssinatura') AS INTEGER) AS norma_ano_assinatura,
        CAST(jget1(j, '$.normaGerada.dataAssinatura') AS DATE) AS norma_data_assinatura,
        CAST(jget1(j, '$.normaGerada.dataPublicacao') AS DATE) AS norma_data_publicacao,

        year AS year_snapshot
    FROM base
)
SELECT *
FROM flat
WHERE id_processo IS NOT NULL;


DROP TABLE IF EXISTS tipo_deliberacao_senado;

CREATE TABLE tipo_deliberacao_senado AS
WITH base AS (
  SELECT
    deliberacao_sigla_tipo   AS sigla_tipo_deliberacao,
    deliberacao_tipo         AS descricao_tipo_deliberacao,
    -- campos auxiliares úteis (opc.)
    deliberacao_id_destino   AS id_destino,
    deliberacao_sigla_destino AS sigla_destino,
    deliberacao_destino      AS destino,
    year_snapshot
  FROM processo_senado
  WHERE deliberacao_sigla_tipo IS NOT NULL
)
SELECT
  sigla_tipo_deliberacao,
  arg_max(descricao_tipo_deliberacao, year_snapshot) AS descricao_tipo_deliberacao,
  arg_max(id_destino,               year_snapshot)   AS id_destino,
  arg_max(sigla_destino,            year_snapshot)   AS sigla_destino,
  arg_max(destino,                  year_snapshot)   AS destino,
  max(year_snapshot)                                AS year_snapshot
FROM base
GROUP BY sigla_tipo_deliberacao;

DROP TABLE IF EXISTS tipo_conteudo_senado;

CREATE TABLE tipo_conteudo_senado AS
WITH base AS (
  SELECT
    id_tipo_conteudo       AS id_tipo_conteudo,      -- chave numérica
    sigla_tipo_conteudo    AS sigla_tipo_conteudo,
    tipo_conteudo          AS descricao_tipo_conteudo,
    tipo_norma_indicada    AS tipo_norma_indicada,   -- pode ser NULL dependendo do caso
    year_snapshot
  FROM processo_senado
  WHERE id_tipo_conteudo IS NOT NULL
)
SELECT
  id_tipo_conteudo,
  arg_max(sigla_tipo_conteudo,    year_snapshot) AS sigla_tipo_conteudo,
  arg_max(descricao_tipo_conteudo,year_snapshot) AS descricao_tipo_conteudo,
  arg_max(tipo_norma_indicada,    year_snapshot) AS tipo_norma_indicada,
  max(year_snapshot)                             AS year_snapshot
FROM base
GROUP BY id_tipo_conteudo;

DROP VIEW IF EXISTS bronze_senado_processo;


Running query in 'duck'

Success


In [28]:
%%sql
SELECT * FROM processo_senado LIMIT 5;

Running query in 'duck'

id_processo,codigo_materia,id_processo_casa_inicial,identificacao,identificacao_processo_inicial,identificacao_externa,ano,casa_identificadora,sigla_casa_iniciadora,sigla_ente_identificador,descricao_sigla,sigla,numero,objetivo,tramitando,id_conteudo,id_tipo_conteudo,sigla_tipo_conteudo,tipo_conteudo,tipo_norma_indicada,ementa,explicacao_ementa,deliberacao_id_destino,deliberacao_sigla_destino,deliberacao_tipo,deliberacao_sigla_tipo,deliberacao_data,deliberacao_destino,id_documento,documento_sigla_tipo,documento_tipo,documento_indexacao,documento_resumo_autoria,documento_data_apresentacao,documento_data_leitura,norma_codigo,norma_numero,norma_sigla_tipo,norma_tipo,norma_descricao,norma_sigla_veiculo,norma_veiculo,norma_numero_int,norma_ano_assinatura,norma_data_assinatura,norma_data_publicacao,year_snapshot
502664,124528,502661,PLC 215/2015,PL 546/2003,{},2015,SF,CD,SF,Projeto de Lei da Câmara,PLC,215,Revisora,Não,502669,11,NORMA_GERAL,Norma Geral,LEI,Estabelece a inclusão do leite na pauta de produtos amparados pela Política de Garantia de Preços Mínimos - PGPM.,Autoriza o Poder Executivo a incluir o leite na pauta dos produtos amparados pela Política de Garantia de Preços Mínimos – PGPM. Beneficia produtores e suas cooperativas.,4,ARQUIVO,Arquivada ao final da Legislatura (art. 332 do RISF),ARQUIVADO_FIM_LEGISLATURA,2022-12-21,Ao arquivo,502670,PROJETO_LEI_ORDINARIA,Projeto de Lei Ordinária,"CRIAÇÃO, LEI FEDERAL, INCLUSÃO, LEITE, POLITICA, GARANTIA, PREÇO MINIMO. AGROPECUARIA, ATIVIDADE PECUARIA, PRODUTOR RURAL.",Câmara dos Deputados,2015-12-17,2015-12-18,None,None,None,None,None,None,None,None,None,None,None,2015
502818,124522,502815,PLC 217/2015,PL 3019/2008,{},2015,SF,CD,SF,Projeto de Lei da Câmara,PLC,217,Revisora,Não,502823,11,NORMA_GERAL,Norma Geral,LEI,"Acrescenta §§ 1º a 4º ao art. 43 da Lei nº 4.591, de 16 de dezembro de 1964, que dispõe sobre o condomínio em edificações e as incorporações imobiliárias.","Altera a Lei nº 4.591/1964, que dispõe sobre o condomínio em edificações e as incorporações imobiliárias, para fixar multa por mês de atraso na entrega do imóvel, considerando-se retardo excessivo no andamento do empreendimento o atraso superior a cento e vinte dias na conclusão da obra.",4,ARQUIVO,Arquivada ao final da Legislatura (art. 332 do RISF),ARQUIVADO_FIM_LEGISLATURA,2022-12-21,Ao arquivo,502824,PROJETO_LEI_ORDINARIA,Projeto de Lei Ordinária,"ALTERAÇÃO, LEI FEDERAL, LEI DOS CONDOMINIOS, INCORPORAÇÃO IMOBILIARIA, ATRASO, ENTREGA, OBRAS, IMOVEL, PAGAMENTO, INDENIZAÇÃO, ALUGUEL. DESTITUIÇÃO, INCORPORADOR.",Câmara dos Deputados,2015-12-17,2015-12-18,None,None,None,None,None,None,None,None,None,None,None,2015
502857,124521,502854,PLC 218/2015,PL 1611/2011,{},2015,SF,CD,SF,Projeto de Lei da Câmara,PLC,218,Revisora,Não,502862,11,NORMA_GERAL,Norma Geral,LEI,"Acrescenta o art. 9º-A à Lei nº 7.347, de 24 de julho de 1985, que “disciplina a ação civil pública de responsabilidade por danos causados ao meio-ambiente, ao consumidor, a bens e direitos de valor artístico, estético, histórico, turístico e paisagístico (VETADO) e dá outras providências”.","Altera a Lei 7.347/1985 que disciplina a ação civil pública de responsabilidade por danos causados ao meio-ambiente, ao consumidor, a bens e direitos de valor artístico, estético, histórico, turístico e paisagístico e dá outras providências, para estabelecer que das decisões ou atos de representante do Ministério Público nos autos de inquérito civil ou em peças informativas poderão ser apresentados recursos ou reclamações ao órgão superior da instituição, que deverão ser resolvidos em quarenta e cinco dias.",4,ARQUIVO,Arquivada ao final da Legislatura (art. 332 do RISF),ARQUIVADO_FIM_LEGISLATURA,2022-12-21,Ao arquivo,502863,PROJETO_LEI_ORDINARIA,Projeto de Lei Ordinária,"ALTERAÇÃO, LEI FEDERAL, LEI DOS INTERESSES DIFUSOS, AÇÃO CIVIL PUBLICA, RECURSO, RECLAMAÇÃO, ATO, DECISÃO, MINISTERIO PUBLICO, INQUERITO.",Câmara dos Deputados,2015-12-17,2015-12-18,None,None,None,None,Non

In [29]:
%%sql
SELECT * FROM tipo_conteudo_senado LIMIT 5;

Running query in 'duck'

id_tipo_conteudo,sigla_tipo_conteudo,descricao_tipo_conteudo,tipo_norma_indicada,year_snapshot
11,NORMA_GERAL,Norma Geral,LEI,2024
167,SUSTACAO_ATO_PODER_EXECUTIVO,Sustação de ato do Poder Executivo,DLG,2024
164,CONCESSAO_SERVICO_TELECOM,Concessão/renovação de serviços de telecomunicação,DLG,2023
3010,NAOCAT_NORMA_JURIDICA,Norma Jurídica - não categorizada,DLG,2020
168,FIXACAO_SUBSIDIO,Fixação de subsídios,DLG,2022


In [30]:
%%sql
SELECT * FROM tipo_deliberacao_senado LIMIT 5;

Running query in 'duck'

sigla_tipo_deliberacao,descricao_tipo_deliberacao,id_destino,sigla_destino,destino,year_snapshot
ARQUIVADO_FIM_LEGISLATURA,Arquivada ao final da Legislatura (art. 332 do RISF),4,ARQUIVO,Ao arquivo,2022
REJEITADO_COMISSAO_TERM,"Rejeitada por Comissão em decisão terminativa (art. 91, § 5º, do RISF)",4,ARQUIVO,Ao arquivo,2023
APROVADA_NO_PLENARIO,Aprovada pelo Plenário,1,CAMARA,À Câmara dos Deputados,2024
APROVADA_EM_COMISSAO_TERMINATIVA,Aprovada por Comissão em decisão terminativa,1,CAMARA,À Câmara dos Deputados,2024
PREJUDICADO,Prejudicada,4,ARQUIVO,Ao arquivo,2024


## Modelos factuais

### Emendas

In [31]:
%%sql
CREATE OR REPLACE VIEW bronze_senado_emendas AS
SELECT * FROM parquet_scan("data/bronze/snapshots/bronze-2020-2024-v1/senado/emendas/year=*/part-*.parquet")
WHERE source = 'senado' AND entity = 'emendas';

SELECT COUNT(*) AS n FROM bronze_senado_emendas;

Running query in 'duck'

n
6463


In [32]:
%%sql
SELECT * FROM bronze_senado_emendas LIMIT 1;

Running query in 'duck'

source,entity,id,url,payload_json,payload_sha256,year
senado,emendas,663587:4497468,https://legis.senado.leg.br/dadosabertos/processo/emenda?idProcesso=663587,"{""autoria"":""Senador Papaléo Paes (PSDB/AP)"",""casa"":""SF"",""codigoColegiado"":834,""dataApresentacao"":""2007-04-16"",""decisoes"":[{""casa"":""SF"",""codigoColegiado"":834,""data"":""2007-04-26"",""descricaoTipo"":""Aprovada"",""idTipo"":1,""nomeColegiado"":""Comissão de Direitos Humanos e Legislação Participativa"",""siglaColegiado"":""CDH""}],""descricaoDocumentoEmenda"":null,""id"":4497468,""idCiEmenda"":4497465,""idCiEmendado"":663592,""idDocumentoEmenda"":4497459,""idProcesso"":663587,""identificacao"":""EMENDA 1 / CDH - PLS 169/2005"",""nomeColegiado"":""Comissão de Direitos Humanos e Legislação Participativa"",""numero"":""1"",""siglaColegiado"":""CDH"",""subemendas"":null,""tipo"":""EMENDA_TOTAL"",""turnoApresentacao"":""NORMAL"",""urlDocumentoEmenda"":""http://legis.senado.leg.br/sdleg-getter/documento?dm=4497459""}",5811ee66646f39590f8870f678d6479321cba3306f863c573896ad433ec32c07,2007


In [33]:
peek_json_payloads(entity='senado/emendas')

[
  {
    "autoria": "Senador Papaléo Paes (PSDB/AP)",
    "casa": "SF",
    "codigoColegiado": 834,
    "dataApresentacao": "2007-04-16",
    "decisoes": [
      {
        "casa": "SF",
        "codigoColegiado": 834,
        "data": "2007-04-26",
        "descricaoTipo": "Aprovada",
        "idTipo": 1,
        "nomeColegiado": "Comissão de Direitos Humanos e Legislação Participativa",
        "siglaColegiado": "CDH"
      }
    ],
    "descricaoDocumentoEmenda": null,
    "id": 4497468,
    "idCiEmenda": 4497465,
    "idCiEmendado": 663592,
    "idDocumentoEmenda": 4497459,
    "idProcesso": 663587,
    "identificacao": "EMENDA 1 / CDH - PLS 169/2005",
    "nomeColegiado": "Comissão de Direitos Humanos e Legislação Participativa",
    "numero": "1",
    "siglaColegiado": "CDH",
    "subemendas": null,
    "tipo": "EMENDA_TOTAL",
    "turnoApresentacao": "NORMAL",
    "urlDocumentoEmenda": "http://legis.senado.leg.br/sdleg-getter/documento?dm=4497459"
  },
  {
    "autoria": "Senador

[{'autoria': 'Senador Papaléo Paes (PSDB/AP)',
  'casa': 'SF',
  'codigoColegiado': 834,
  'dataApresentacao': '2007-04-16',
  'decisoes': [{'casa': 'SF',
    'codigoColegiado': 834,
    'data': '2007-04-26',
    'descricaoTipo': 'Aprovada',
    'idTipo': 1,
    'nomeColegiado': 'Comissão de Direitos Humanos e Legislação Participativa',
    'siglaColegiado': 'CDH'}],
  'descricaoDocumentoEmenda': None,
  'id': 4497468,
  'idCiEmenda': 4497465,
  'idCiEmendado': 663592,
  'idDocumentoEmenda': 4497459,
  'idProcesso': 663587,
  'identificacao': 'EMENDA 1 / CDH - PLS 169/2005',
  'nomeColegiado': 'Comissão de Direitos Humanos e Legislação Participativa',
  'numero': '1',
  'siglaColegiado': 'CDH',
  'subemendas': None,
  'tipo': 'EMENDA_TOTAL',
  'turnoApresentacao': 'NORMAL',
  'urlDocumentoEmenda': 'http://legis.senado.leg.br/sdleg-getter/documento?dm=4497459'},
 {'autoria': 'Senador Demóstenes Torres (DEM/GO)',
  'casa': 'SF',
  'codigoColegiado': 34,
  'dataApresentacao': '2009-05-21'

In [34]:
%%sql

DROP TABLE IF EXISTS emendas_senado;

CREATE TABLE emendas_senado AS
WITH base AS (
    SELECT TRY_CAST(payload_json AS JSON) AS j, year
    FROM bronze_senado_emendas
),
flat AS (
    SELECT
        -- chaves e metadados
        CAST(jget1(j, '$.id')                AS BIGINT)  AS id_emenda,
        CAST(jget1(j, '$.idCiEmenda')        AS BIGINT)  AS id_ci_emenda,
        CAST(jget1(j, '$.idCiEmendado')      AS BIGINT)  AS id_ci_emendado,
        CAST(jget1(j, '$.idDocumentoEmenda') AS BIGINT)  AS id_documento_emenda,
        CAST(jget1(j, '$.idProcesso')        AS BIGINT)  AS id_processo,

        -- identificação/descrição
        jget1(j, '$.identificacao')                      AS identificacao,
        CAST(jget1(j, '$.numero') AS INTEGER)            AS numero,
        jget1(j, '$.autoria')                             AS autoria,
        jget1(j, '$.descricaoDocumentoEmenda')            AS descricao_documento_emenda,
        jget1(j, '$.tipo')                                AS tipo_emenda,
        jget1(j, '$.turnoApresentacao')                   AS turno_apresentacao,

        -- casa/colegiado
        jget1(j, '$.casa')                                AS casa,
        CAST(jget1(j, '$.codigoColegiado') AS BIGINT)     AS codigo_colegiado,
        jget1(j, '$.siglaColegiado')                      AS sigla_colegiado,
        jget1(j, '$.nomeColegiado')                       AS nome_colegiado,

        -- documentos e datas
        CAST(jget1(j, '$.dataApresentacao') AS DATE)      AS data_apresentacao,
        jget1(j, '$.urlDocumentoEmenda')                  AS url_documento_emenda,

        -- snapshot
        year AS year_snapshot
    FROM base
)
SELECT *
FROM flat
WHERE id_emenda IS NOT NULL;


Running query in 'duck'

Count
6463


In [35]:
%%sql

DROP TABLE IF EXISTS tipo_emendas_senado;

CREATE TABLE tipo_emendas_senado AS
WITH base AS (
    SELECT
        tipo_emenda,
        year_snapshot
    FROM emendas_senado
    WHERE tipo_emenda IS NOT NULL
)
SELECT
    tipo_emenda,
    max(year_snapshot) AS year_snapshot
FROM base
GROUP BY tipo_emenda;


Running query in 'duck'

Count
2


In [36]:
%%sql
DROP VIEW IF EXISTS bronze_senado_emendas;

Running query in 'duck'

Success


### Relatorias

In [37]:
%%sql
CREATE OR REPLACE VIEW bronze_senado_relatorias AS
SELECT * FROM parquet_scan("data/bronze/snapshots/bronze-2020-2024-v1/senado/relatorias/year=*/part-*.parquet")
WHERE source = 'senado' AND entity = 'relatorias';

SELECT COUNT(*) AS n FROM bronze_senado_relatorias;

Running query in 'duck'

n
2029


In [38]:
%%sql
SELECT * FROM bronze_senado_relatorias LIMIT 5;

Running query in 'duck'

source,entity,id,url,payload_json,payload_sha256,year
senado,relatorias,663587:7701594,https://legis.senado.leg.br/dadosabertos/processo/relatoria?idProcesso=663587,"{""autoriaProcesso"":""Senador Paulo Paim (PT/RS)"",""casaRelator"":""SF"",""codigoColegiado"":834,""codigoMateria"":73682,""codigoParlamentar"":53,""codigoTipoColegiado"":21,""dataApresentacaoProcesso"":""2005-05-12 00:00:00"",""dataDesignacao"":""2005-05-17 00:00:00"",""dataDestituicao"":""2006-05-24 00:00:00"",""dataFimColegiado"":null,""descricaoTipoEncerramento"":""Redistribuição"",""descricaoTipoRelator"":""Relator"",""emailParlamentar"":""leomar@senador.leg.br"",""ementaProcesso"":""Altera dispositivo da Lei nº 10.741, de 1º de outubro de 2003, que dispõe sobre o Estatuto do Idoso e dá outras providências."",""formaTratamentoParlamentar"":""Senador"",""id"":7701594,""idProcesso"":663587,""idTipoRelator"":1,""identificacaoProcesso"":""PLS 169/2005"",""nomeColegiado"":""Comissão de Direitos Humanos e Legislação Participativa"",""nomeCompleto"":""Leomar de Melo Quintanilha"",""nomeParlamentar"":""Leomar Quintanilha"",""numeroAutuacao"":""1"",""sexoParlamentar"":""M"",""siglaCasa"":""SF"",""siglaColegiado"":""CDH"",""siglaPartidoParlamentar"":""MDB"",""tramitando"":""N"",""ufParlamentar"":""TO"",""urlFotoParlamentar"":""http://www.senado.leg.br/senadores/img/fotos-oficiais/senador53.jpg"",""urlPaginaParlamentar"":""http://www25.senado.leg.br/web/senadores/senador/-/perfil/53""}",9292143e0761e4ff534bffc1f66e313a48ed920150614b63b26e85033fae7c3b,2005
senado,relatorias,663587:7692977,https://legis.senado.leg.br/dadosabertos/processo/relatoria?idProcesso=663587,"{""autoriaProcesso"":""Senador Paulo Paim (PT/RS)"",""casaRelator"":""SF"",""codigoColegiado"":834,""codigoMateria"":73682,""codigoParlamentar"":3,""codigoTipoColegiado"":21,""dataApresentacaoProcesso"":""2005-05-12 00:00:00"",""dataDesignacao"":""2006-05-31 00:00:00"",""dataDestituicao"":""2007-02-07 00:00:00"",""dataFimColegiado"":null,""descricaoTipoEncerramento"":""Redistribuição"",""descricaoTipoRelator"":""Relator"",""emailParlamentar"":""antoniocarlosvaladares@senador.leg.br"",""ementaProcesso"":""Altera dispositivo da Lei nº 10.741, de 1º de outubro de 2003, que dispõe sobre o Estatuto do Idoso e dá outras providências."",""formaTratamentoParlamentar"":""Senador"",""id"":7692977,""idProcesso"":663587,""idTipoRelator"":1,""identificacaoProcesso"":""PLS 169/2005"",""nomeColegiado"":""Comissão de Direitos Humanos e Legislação Participativa"",""nomeCompleto"":""Antonio Carlos Valadares"",""nomeParlamentar"":""Antonio Carlos Valadares"",""numeroAutuacao"":""1"",""sexoParlamentar"":""M"",""siglaCasa"":""SF"",""siglaColegiado"":""CDH"",""siglaPartidoParlamentar"":""PSB"",""tramitando"":""N"",""ufParlamentar"":""SE"",""urlFotoParlamentar"":""http://www.senado.leg.br/senadores/img/fotos-oficiais/senador3.jpg"",""urlPaginaParlamentar"":""http://www25.senado.leg.br/web/senadores/senador/-/perfil/3""}",bb3baeef74c7fb269c7da58af21012ef898428326ccc3dfd10365377f376afa8,2006
senado,relatorias,663587:7698040,https://legis.senado.leg.br/dadosabertos/processo/relatoria?idProcesso=663587,"{""autoriaProcesso"":""Senador Paulo Paim (PT/RS)"",""casaRelator"":""SF"",""codigoColegiado"":834,""codigoMateria"":73682,""codigoParlamentar"":345,""codigoTipoColegiado"":21,""dataApresentacaoProcesso"":""2005-05-12 00:00:00"",""dataDesignacao"":""2007-04-26 00:00:00"",""dataDestituicao"":""2007-05-10 00:00:00"",""dataFimColegiado"":null,""descricaoTipoEncerramento"":""Parecer Oferecido"",""descricaoTipoRelator"":""Relator"",""emailParlamentar"":""sen.flavioarns@senado.leg.br"",""ementaProcesso"":""Altera dispositivo da Lei nº 10.741, de 1º de outubro de 2003, que dispõe sobre o Estatuto do Idoso e dá outras providências."",""formaTratamentoParlamentar"":""Senador"",""id"":7698040,""idProcesso"":663587,""idTipoRelator"":1,""identificacaoProcesso"":""PLS 169/2005"",""nomeColegiado"":""Comissão de Direitos Humanos e Legislação Particip

In [39]:
peek_json_payloads(entity="senado/relatorias")

[
  {
    "autoriaProcesso": "Senador Paulo Paim (PT/RS)",
    "casaRelator": "SF",
    "codigoColegiado": 834,
    "codigoMateria": 73682,
    "codigoParlamentar": 53,
    "codigoTipoColegiado": 21,
    "dataApresentacaoProcesso": "2005-05-12 00:00:00",
    "dataDesignacao": "2005-05-17 00:00:00",
    "dataDestituicao": "2006-05-24 00:00:00",
    "dataFimColegiado": null,
    "descricaoTipoEncerramento": "Redistribuição",
    "descricaoTipoRelator": "Relator",
    "emailParlamentar": "leomar@senador.leg.br",
    "ementaProcesso": "Altera dispositivo da Lei nº 10.741, de 1º de outubro de 2003, que dispõe sobre o Estatuto do Idoso e dá outras providências.",
    "formaTratamentoParlamentar": "Senador",
    "id": 7701594,
    "idProcesso": 663587,
    "idTipoRelator": 1,
    "identificacaoProcesso": "PLS 169/2005",
    "nomeColegiado": "Comissão de Direitos Humanos e Legislação Participativa",
    "nomeCompleto": "Leomar de Melo Quintanilha",
    "nomeParlamentar": "Leomar Quintanilha",


[{'autoriaProcesso': 'Senador Paulo Paim (PT/RS)',
  'casaRelator': 'SF',
  'codigoColegiado': 834,
  'codigoMateria': 73682,
  'codigoParlamentar': 53,
  'codigoTipoColegiado': 21,
  'dataApresentacaoProcesso': '2005-05-12 00:00:00',
  'dataDesignacao': '2005-05-17 00:00:00',
  'dataDestituicao': '2006-05-24 00:00:00',
  'dataFimColegiado': None,
  'descricaoTipoEncerramento': 'Redistribuição',
  'descricaoTipoRelator': 'Relator',
  'emailParlamentar': 'leomar@senador.leg.br',
  'ementaProcesso': 'Altera dispositivo da Lei nº 10.741, de 1º de outubro de 2003, que dispõe sobre o Estatuto do Idoso e dá outras providências.',
  'formaTratamentoParlamentar': 'Senador',
  'id': 7701594,
  'idProcesso': 663587,
  'idTipoRelator': 1,
  'identificacaoProcesso': 'PLS 169/2005',
  'nomeColegiado': 'Comissão de Direitos Humanos e Legislação Participativa',
  'nomeCompleto': 'Leomar de Melo Quintanilha',
  'nomeParlamentar': 'Leomar Quintanilha',
  'numeroAutuacao': '1',
  'sexoParlamentar': 'M',

In [40]:
%%sql
DROP TABLE IF EXISTS relatorias_senado;

CREATE TABLE relatorias_senado AS
WITH base AS (
    SELECT TRY_CAST(payload_json AS JSON) AS j, year
    FROM bronze_senado_relatorias
)
SELECT
    -- chaves e relacionamentos
    CAST(jget1(j, '$.id')                AS BIGINT)   AS id_relatoria,
    CAST(jget1(j, '$.idProcesso')        AS BIGINT)   AS id_processo,
    CAST(jget1(j, '$.codigoMateria')     AS BIGINT)   AS codigo_materia,
    CAST(jget1(j, '$.codigoParlamentar') AS BIGINT)   AS codigo_parlamentar,

    -- colegiado
    CAST(jget1(j, '$.codigoColegiado')      AS BIGINT)  AS codigo_colegiado,
    CAST(jget1(j, '$.codigoTipoColegiado')  AS BIGINT)  AS codigo_tipo_colegiado,
    jget1(j, '$.siglaColegiado')                         AS sigla_colegiado,
    jget1(j, '$.nomeColegiado')                          AS nome_colegiado,

    -- processo
    jget1(j, '$.autoriaProcesso')                        AS autoria_processo,
    jget1(j, '$.identificacaoProcesso')                  AS identificacao_processo,
    jget1(j, '$.ementaProcesso')                         AS ementa_processo,
    CAST(jget1(j, '$.numeroAutuacao') AS INTEGER)        AS numero_autuacao,
    jget1(j, '$.tramitando')                              AS tramitando,

    -- casa/relator
    jget1(j, '$.siglaCasa')                               AS sigla_casa,
    jget1(j, '$.casaRelator')                             AS casa_relator,
    jget1(j, '$.descricaoTipoRelator')                    AS descricao_tipo_relator,
    CAST(jget1(j, '$.idTipoRelator') AS INTEGER)          AS id_tipo_relator,
    jget1(j, '$.descricaoTipoEncerramento')               AS descricao_tipo_encerramento,

    -- parlamentar
    jget1(j, '$.formaTratamentoParlamentar')              AS forma_tratamento_parlamentar,
    jget1(j, '$.nomeParlamentar')                         AS nome_parlamentar,
    jget1(j, '$.nomeCompleto')                            AS nome_completo,
    jget1(j, '$.siglaPartidoParlamentar')                 AS sigla_partido_parlamentar,
    jget1(j, '$.ufParlamentar')                           AS uf_parlamentar,
    jget1(j, '$.sexoParlamentar')                         AS sexo_parlamentar,
    jget1(j, '$.emailParlamentar')                        AS email_parlamentar,
    jget1(j, '$.urlFotoParlamentar')                      AS url_foto_parlamentar,
    jget1(j, '$.urlPaginaParlamentar')                    AS url_pagina_parlamentar,

    -- datas
    CAST(jget1(j, '$.dataApresentacaoProcesso') AS DATETIME) AS data_apresentacao_processo,
    CAST(jget1(j, '$.dataDesignacao')           AS DATETIME) AS data_designacao,
    CAST(jget1(j, '$.dataDestituicao')          AS DATETIME) AS data_destituicao,
    CAST(jget1(j, '$.dataFimColegiado')         AS DATETIME) AS data_fim_colegiado,

    -- snapshot
    year AS year_snapshot
FROM base
WHERE jget1(j, '$.id') IS NOT NULL;
DROP VIEW IF EXISTS bronze_senado_relatorias;

Running query in 'duck'

Success


In [41]:
%%sql
SELECT * FROM relatorias_senado LIMIT 5;

Running query in 'duck'

id_relatoria,id_processo,codigo_materia,codigo_parlamentar,codigo_colegiado,codigo_tipo_colegiado,sigla_colegiado,nome_colegiado,autoria_processo,identificacao_processo,ementa_processo,numero_autuacao,tramitando,sigla_casa,casa_relator,descricao_tipo_relator,id_tipo_relator,descricao_tipo_encerramento,forma_tratamento_parlamentar,nome_parlamentar,nome_completo,sigla_partido_parlamentar,uf_parlamentar,sexo_parlamentar,email_parlamentar,url_foto_parlamentar,url_pagina_parlamentar,data_apresentacao_processo,data_designacao,data_destituicao,data_fim_colegiado,year_snapshot
7701594,663587,73682,53,834,21,CDH,Comissão de Direitos Humanos e Legislação Participativa,Senador Paulo Paim (PT/RS),PLS 169/2005,"Altera dispositivo da Lei nº 10.741, de 1º de outubro de 2003, que dispõe sobre o Estatuto do Idoso e dá outras providências.",1,N,SF,SF,Relator,1,Redistribuição,Senador,Leomar Quintanilha,Leomar de Melo Quintanilha,MDB,TO,M,leomar@senador.leg.br,http://www.senado.leg.br/senadores/img/fotos-oficiais/senador53.jpg,http://www25.senado.leg.br/web/senadores/senador/-/perfil/53,2005-05-12 00:00:00,2005-05-17 00:00:00,2006-05-24 00:00:00,None,2005
7692977,663587,73682,3,834,21,CDH,Comissão de Direitos Humanos e Legislação Participativa,Senador Paulo Paim (PT/RS),PLS 169/2005,"Altera dispositivo da Lei nº 10.741, de 1º de outubro de 2003, que dispõe sobre o Estatuto do Idoso e dá outras providências.",1,N,SF,SF,Relator,1,Redistribuição,Senador,Antonio Carlos Valadares,Antonio Carlos Valadares,PSB,SE,M,antoniocarlosvaladares@senador.leg.br,http://www.senado.leg.br/senadores/img/fotos-oficiais/senador3.jpg,http://www25.senado.leg.br/web/senadores/senador/-/perfil/3,2005-05-12 00:00:00,2006-05-31 00:00:00,2007-02-07 00:00:00,None,2006
7698040,663587,73682,345,834,21,CDH,Comissão de Direitos Humanos e Legislação Participativa,Senador Paulo Paim (PT/RS),PLS 169/2005,"Altera dispositivo da Lei nº 10.741, de 1º de outubro de 2003, que dispõe sobre o Estatuto do Idoso e dá outras providências.",1,N,SF,SF,Relator,1,Parecer Oferecido,Senador,Flávio Arns,Flávio José Arns,PT,PR,M,sen.flavioarns@senado.leg.br,http://www.senado.leg.br/senadores/img/fotos-oficiais/senador345.jpg,http://www25.senado.leg.br/web/senadores/senador/-/perfil/345,2005-05-12 00:00:00,2007-04-26 00:00:00,2007-05-10 00:00:00,None,2007
7698764,663587,73682,3393,834,21,CDH,Comissão de Direitos Humanos e Legislação Participativa,Senador Paulo Paim (PT/RS),PLS 169/2005,"Altera dispositivo da Lei nº 10.741, de 1º de outubro de 2003, que dispõe sobre o Estatuto do Idoso e dá outras providências.",1,N,SF,SF,Relator,1,"Substituído por ""ad hoc""",Senador,Papaléo Paes,João Bosco Papaléo Paes,PSDB,AP,M,gab.papaleopaes@senado.leg.br,http://www.senado.leg.br/senadores/img/fotos-oficiais/senador3393.jpg,http://www25.senado.leg.br/web/senadores/senador/-/perfil/3393,2005-05-12 00:00:00,2007-02-08 00:00:00,2007-04-26 00:00:00,None,2007
7692575,2970890,86332,3372,40,21,CAS,Comissão de Assuntos Sociais,Senador Alvaro Dias (PSDB/PR),PLS 260/2008,"Altera o § 1º do art. 12 da Lei nº 8.212, de 24 de julho de 1991, e o § 1º do art. 11 da Lei nº 8.213, de 24 de julho de 1991, acrescentando-lhe § 6º, para permitir a contratação eventual de empregados, pelos segurados especiais, e dá outras providências.",1,N,SF,SF,Relator,1,Fim de Legislatura,Senador,Valdir Raupp,Valdir Raupp de Matos,MDB,RO,M,valdir.raupp@senador.leg.br,http://www.senado.leg.br/senadores/img/fotos-oficiais/senador3372.jpg,http://www25.senado.leg.br/web/senadores/senador/-/perfil/3372,2008-07-01 00:00:00,2008-12-03 00:00:00,2010-12-22 00:00:00,None,2008


### Votações e votos

In [42]:
%%sql
CREATE OR REPLACE VIEW bronze_senado_votacoes AS
SELECT * FROM parquet_scan("data/bronze/snapshots/bronze-2020-2024-v1/senado/votacoes/year=*/part-*.parquet")
WHERE source = 'senado' AND entity = 'votacoes';

SELECT COUNT(*) AS n FROM bronze_senado_votacoes;

Running query in 'duck'

n
604


In [43]:
%%sql
SELECT * FROM bronze_senado_votacoes LIMIT 1;

Running query in 'duck'

+--------+----------+--------------+-----------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [44]:
peek_json_payloads(entity="senado/votacoes")

[
  {
    "ano": 2015,
    "casaSessao": "SF",
    "codigoMateria": 121994,
    "codigoSessao": 22254,
    "codigoSessaoLegislativa": 850,
    "codigoSessaoVotacao": 5481,
    "codigoVotacaoSve": null,
    "dataApresentacao": "2015-06-25T00:00:00",
    "dataSessao": "2015-10-06T00:00:00",
    "descricaoVotacao": "Votação nominal da Emenda nº 15-CCJ (Substituvivo)",
    "ementa": "Acrescenta o art. 166-A à Constituição Federal, para dispor sobre a Autoridade Fiscal Independente.",
    "idProcesso": 1399048,
    "identificacao": "PEC 83/2015",
    "informeLegislativo": {
      "casaColegiado": null,
      "casaEnteAdm": "SF",
      "codigoColegiado": null,
      "data": "2015-10-06T00:00:01",
      "id": 1881857,
      "idEnteAdm": 7217796,
      "idEvento": 9815176,
      "nomeColegiado": null,
      "nomeEnteAdm": "SUBSECRETARIA DE ATA - PLENÁRIO",
      "numeroAutuacao": 1,
      "siglaColegiado": null,
      "siglaEnteAdm": "ATA-PLEN",
      "texto": "Anunciada a votação da matéria, 

[{'ano': 2015,
  'casaSessao': 'SF',
  'codigoMateria': 121994,
  'codigoSessao': 22254,
  'codigoSessaoLegislativa': 850,
  'codigoSessaoVotacao': 5481,
  'codigoVotacaoSve': None,
  'dataApresentacao': '2015-06-25T00:00:00',
  'dataSessao': '2015-10-06T00:00:00',
  'descricaoVotacao': 'Votação nominal da Emenda nº 15-CCJ (Substituvivo)',
  'ementa': 'Acrescenta o art. 166-A à Constituição Federal, para dispor sobre a Autoridade Fiscal Independente.',
  'idProcesso': 1399048,
  'identificacao': 'PEC 83/2015',
  'informeLegislativo': {'casaColegiado': None,
   'casaEnteAdm': 'SF',
   'codigoColegiado': None,
   'data': '2015-10-06T00:00:01',
   'id': 1881857,
   'idEnteAdm': 7217796,
   'idEvento': 9815176,
   'nomeColegiado': None,
   'nomeEnteAdm': 'SUBSECRETARIA DE ATA - PLENÁRIO',
   'numeroAutuacao': 1,
   'siglaColegiado': None,
   'siglaEnteAdm': 'ATA-PLEN',
   'texto': 'Anunciada a votação da matéria, em primeiro turno, usam da palavra o Senador José Serra, Relator, e o Senador

In [45]:
%%sql
DROP TABLE IF EXISTS votacoes_senado;

CREATE TABLE votacoes_senado AS
WITH base AS (
    SELECT TRY_CAST(payload_json AS JSON) AS j, year AS year_snapshot
    FROM bronze_senado_votacoes
)
SELECT
    CAST(jget1(j, '$.codigoSessaoVotacao') AS BIGINT) AS id_votacao,
    CAST(jget1(j, '$.codigoMateria') AS BIGINT) AS id_materia,
    CAST(jget1(j, '$.idProcesso') AS BIGINT) AS id_processo,
    jget1(j, '$.identificacao') AS identificacao,

    jget1(j, '$.sigla') AS sigla,
    jget1(j, '$.numero') AS numero,
    CAST(jget1(j, '$.ano') AS INTEGER) AS ano,

    CAST(jget1(j, '$.codigoSessao') AS BIGINT) AS codigo_sessao,
    CAST(jget1(j, '$.numeroSessao') AS INTEGER) AS numero_sessao,
    CAST(jget1(j, '$.sequencialSessao') AS INTEGER) AS sequencial_sessao,
    jget1(j, '$.siglaTipoSessao') AS sigla_tipo_sessao,
    jget1(j, '$.casaSessao') AS casa_sessao,
    CAST(jget1(j, '$.codigoSessaoLegislativa') AS BIGINT) AS codigo_sessao_legislativa,

    CAST(jget1(j, '$.dataApresentacao') AS TIMESTAMP) AS data_apresentacao,
    CAST(jget1(j, '$.dataSessao') AS TIMESTAMP) AS data_sessao,

    jget1(j, '$.descricaoVotacao') AS descricao_votacao,
    jget1(j, '$.ementa') AS ementa,

    jget1(j, '$.resultadoVotacao') AS resultado_votacao,
    CAST(jget1(j, '$.totalVotosSim') AS INTEGER) AS total_votos_sim,
    CAST(jget1(j, '$.totalVotosNao') AS INTEGER) AS total_votos_nao,
    CAST(jget1(j, '$.totalVotosAbstencao') AS INTEGER) AS total_votos_abstencao,
    jget1(j, '$.votacaoSecreta') AS votacao_secreta,

    -- informe legislativo expandido
    CAST(jget1(j, '$.informeLegislativo.id') AS BIGINT) AS id_informe,
    CAST(jget1(j, '$.informeLegislativo.idEvento') AS BIGINT) AS id_evento,
    CAST(jget1(j, '$.informeLegislativo.codigoColegiado') AS BIGINT) AS codigo_colegiado,
    jget1(j, '$.informeLegislativo.nomeColegiado') AS nome_colegiado,
    jget1(j, '$.informeLegislativo.siglaColegiado') AS sigla_colegiado,
    CAST(jget1(j, '$.informeLegislativo.data') AS TIMESTAMP) AS data_informe,
    jget1(j, '$.informeLegislativo.texto') AS texto_informe,

    year_snapshot
FROM base;


Running query in 'duck'

Count
604


In [46]:
%%sql
SELECT * FROM votacoes_senado LIMIT 1;

Running query in 'duck'

id_votacao,id_materia,id_processo,identificacao,sigla,numero,ano,codigo_sessao,numero_sessao,sequencial_sessao,sigla_tipo_sessao,casa_sessao,codigo_sessao_legislativa,data_apresentacao,data_sessao,descricao_votacao,ementa,resultado_votacao,total_votos_sim,total_votos_nao,total_votos_abstencao,votacao_secreta,id_informe,id_evento,codigo_colegiado,nome_colegiado,sigla_colegiado,data_informe,texto_informe,year_snapshot
5481,121994,1399048,PEC 83/2015,PEC,83,2015,22254,176,3,DOR,SF,850,2015-06-25 00:00:00,2015-10-06 00:00:00,Votação nominal da Emenda nº 15-CCJ (Substituvivo),"Acrescenta o art. 166-A à Constituição Federal, para dispor sobre a Autoridade Fiscal Independente.",None,None,None,None,N,1881857,9815176,None,None,None,2015-10-06 00:00:01,"Anunciada a votação da matéria, em primeiro turno, usam da palavra o Senador José Serra, Relator, e o Senador Delcídio do Amaral; tendo o Relator proposto Adendo ao Substitutivo da CCJ.A seguir, usam da palavra os Senadores Lindbergh Farias, Roberto Requião, Randolfe Rodrigues, Vanessa Grazziotin, Waldemir Moka, Blairo Maggi, Telmário Mota, Lúcia Vânia, Rose de Freitas e Magno Malta. Rejeitado, em primeiro turno, o Substitutivo (Emenda nº 15-CCJ), com o Adendo proposto pelo Senador José Serra, Relator, nesta oportunidade, com o seguinte resultado: Sim 40, Não 19, Abst. 02, Presidente 01, Total 62.",2015


In [47]:
%%sql
DROP TABLE IF EXISTS votos_senado;

CREATE TABLE votos_senado AS
WITH base AS (
  SELECT
    TRY_CAST(payload_json AS JSON) AS j,
    year AS year_snapshot
  FROM bronze_senado_votacoes
),
exploded AS (
  SELECT
    CAST(jget1(b.j, '$.codigoVotacaoSve') AS BIGINT)        AS codigo_votacao_sve,
    CAST(jget1(b.j, '$.codigoSessaoVotacao') AS BIGINT)     AS codigo_sessao_votacao,
    CAST(jget1(b.j, '$.codigoMateria') AS BIGINT)           AS codigo_materia,
    jget1(b.j, '$.identificacao')                           AS identificacao_materia,

    -- Campos do voto
    CAST(json_extract_string(e.value, '$.codigoParlamentar') AS BIGINT) AS codigo_parlamentar,
    json_extract_string(e.value, '$.nomeParlamentar')        AS nome_parlamentar,
    json_extract_string(e.value, '$.sexoParlamentar')        AS sexo_parlamentar,
    json_extract_string(e.value, '$.siglaPartidoParlamentar')AS sigla_partido_parlamentar,
    json_extract_string(e.value, '$.siglaUFParlamentar')     AS sigla_uf_parlamentar,
    json_extract_string(e.value, '$.siglaVotoParlamentar')   AS sigla_voto_parlamentar,
    json_extract_string(e.value, '$.descricaoVotoParlamentar') AS descricao_voto_parlamentar,

    -- Snapshot
    b.year_snapshot
  FROM base b
  -- explode apenas quando existir a lista
  CROSS JOIN json_each(b.j, '$.votos') AS e
),
numbered AS (
    SELECT
        ROW_NUMBER() OVER () AS id_voto,
        codigo_votacao_sve,
        codigo_sessao_votacao,
        codigo_materia,
        identificacao_materia,
        codigo_parlamentar,
        nome_parlamentar,
        sexo_parlamentar,
        sigla_partido_parlamentar,
        sigla_uf_parlamentar,
        sigla_voto_parlamentar,
        descricao_voto_parlamentar,
        year_snapshot
    FROM exploded
)
SELECT * FROM numbered;


Running query in 'duck'

Count
48877


In [48]:
%%sql
SELECT * FROM votos_senado LIMIT 5;

Running query in 'duck'

id_voto,codigo_votacao_sve,codigo_sessao_votacao,codigo_materia,identificacao_materia,codigo_parlamentar,nome_parlamentar,sexo_parlamentar,sigla_partido_parlamentar,sigla_uf_parlamentar,sigla_voto_parlamentar,descricao_voto_parlamentar,year_snapshot
1,None,5481,121994,PEC 83/2015,4697,Ângela Portela,F,PT,RR,Sim,None,2015
2,None,5554,123909,PEC 143/2015,4697,Ângela Portela,F,PT,RR,Sim,None,2016
3,None,5708,128035,PEC 2/2017,4697,Ângela Portela,F,PDT,RR,Sim,None,2017
4,2725,5960,127414,PEC 57/2016,3806,Zequinha Marinho,M,PSC,PA,Sim,None,2019
5,2778,5965,129887,PEC 26/2017,3806,Zequinha Marinho,M,PSC,PA,MIS,Missão da Casa no País/exterior,2019


In [49]:
%%sql

DROP VIEW IF EXISTS bronze_senado_votacoes;

Running query in 'duck'

Success


### Votações / Colegiado

## Outras relações entre tabelas

### Processos e autoria iniciativa

In [50]:
%%sql
CREATE OR REPLACE VIEW bronze_senado_processo AS
SELECT * FROM parquet_scan("data/bronze/snapshots/bronze-2020-2024-v1/senado/processo/details/year=*/part-*.parquet")
WHERE source = 'senado' AND entity = 'processo';

SELECT COUNT(*) AS n FROM bronze_senado_processo;

Running query in 'duck'

n
19809


In [51]:
%%sql
DROP TABLE IF EXISTS autoria_iniciativa_senado;

CREATE TABLE autoria_iniciativa_senado AS
WITH base AS (
    SELECT
        TRY_CAST(payload_json AS JSON) AS j,
        year AS year_snapshot,
        CAST(jget1(payload_json, '$.id') AS BIGINT) AS id_processo
    FROM bronze_senado_processo
),
exploded AS (
    SELECT
        b.id_processo,
        CAST(json_extract_string(e.value, '$.idEnte') AS BIGINT) AS id_ente,
        json_extract_string(e.value, '$.autor') AS autor,
        json_extract_string(e.value, '$.casaEnte') AS casa_ente,
        CAST(json_extract_string(e.value, '$.codigoParlamentar') AS BIGINT) AS codigo_parlamentar,
        json_extract_string(e.value, '$.descricaoTipo') AS descricao_tipo,
        json_extract_string(e.value, '$.ente') AS ente,
        CAST(json_extract_string(e.value, '$.ordem') AS INTEGER) AS ordem,
        json_extract_string(e.value, '$.outrosAutoresNaoInformados') AS outros_autores_nao_informados,
        json_extract_string(e.value, '$.siglaEnte') AS sigla_ente,
        json_extract_string(e.value, '$.siglaTipo') AS sigla_tipo,
        b.year_snapshot
    FROM base b
    CROSS JOIN json_each(b.j, '$.autoriaIniciativa') AS e
),
numbered AS (
    SELECT
        ROW_NUMBER() OVER () AS id_autoria_iniciativa,
        id_processo,
        codigo_parlamentar,
        descricao_tipo,
        ente,
        ordem,
        outros_autores_nao_informados,
        sigla_ente,
        sigla_tipo,
        year_snapshot
    FROM exploded
)
SELECT *
FROM numbered;


Running query in 'duck'

Count
35736


In [52]:
%%sql
SELECT * FROM autoria_iniciativa_senado LIMIT 5;

Running query in 'duck'

id_autoria_iniciativa,id_processo,codigo_parlamentar,descricao_tipo,ente,ordem,outros_autores_nao_informados,sigla_ente,sigla_tipo,year_snapshot
1,2526458,None,PRESIDENTE_REPUBLICA,Presidência da República,1,Não,PR,PRESIDENTE_REPUBLICA,2001
2,663587,825,SENADOR,Senado Federal,1,Não,SF,SENADOR,2005
3,1066060,None,COMISSAO_CAMARA,"Comissão de Ciência e Tecnologia, Comunicação e Informática",1,Não,CCTCI,COMISSAO_CAMARA,2007
4,2978567,825,SENADOR,Senado Federal,1,Não,SF,SENADOR,2007
5,2968939,825,SENADOR,Senado Federal,1,Não,SF,SENADOR,2008


In [53]:
%%sql
DROP TABLE IF EXISTS situacoes_senado;

CREATE TABLE situacoes_senado AS
WITH base AS (
    SELECT
        TRY_CAST(payload_json AS JSON) AS j,
        year AS year_snapshot,
        CAST(json_extract_string(payload_json, '$.id') AS BIGINT) AS id_processo
    FROM bronze_senado_processo
),
autuacoes AS (
    SELECT
        b.id_processo,
        CAST(json_extract_string(a.value, '$.numero') AS INTEGER) AS numero_autuacao,
        a.value AS autuacao_json,
        b.year_snapshot
    FROM base b
    CROSS JOIN json_each(b.j, '$.autuacoes') AS a
),
situacoes AS (
    SELECT
        a.id_processo,
        a.numero_autuacao,
        CAST(json_extract_string(s.value, '$.idTipo') AS INTEGER) AS id_tipo_situacao,
        json_extract_string(s.value, '$.sigla') AS sigla_situacao,
        json_extract_string(s.value, '$.descricao') AS descricao_situacao,
        TRY_CAST(json_extract_string(s.value, '$.inicio') AS DATE) AS data_inicio,
        TRY_CAST(json_extract_string(s.value, '$.fim') AS DATE) AS data_fim,
        a.year_snapshot
    FROM autuacoes a
    CROSS JOIN json_each(a.autuacao_json, '$.situacoes') AS s
),
numbered AS (
    SELECT
        ROW_NUMBER() OVER () AS id_situacao,
        id_processo,
        numero_autuacao,
        id_tipo_situacao,
        sigla_situacao,
        descricao_situacao,
        data_inicio,
        data_fim,
        year_snapshot
    FROM situacoes
)
SELECT *
FROM numbered;


Running query in 'duck'

Count
93320


In [54]:
%%sql
DROP TABLE IF EXISTS despachos_senado;

CREATE TABLE despachos_senado AS
WITH base AS (
    SELECT
        TRY_CAST(payload_json AS JSON) AS j,
        year AS year_snapshot,
        CAST(json_extract_string(payload_json, '$.id') AS BIGINT) AS id_processo
    FROM bronze_senado_processo
),
exploded AS (
    SELECT
        b.id_processo,
        CAST(json_extract_string(d.value, '$.id') AS BIGINT) AS id_despacho,
        TRY_CAST(json_extract_string(d.value, '$.data') AS DATE) AS data_despacho,
        json_extract_string(d.value, '$.cancelado') AS cancelado,
        json_extract_string(d.value, '$.tipoMotivacao') AS tipo_motivacao,
        json_extract_string(d.value, '$.siglaTipoMotivacao') AS sigla_tipo_motivacao,
        b.year_snapshot
    FROM base b
    CROSS JOIN json_each(b.j, '$.despachos') AS d
)
SELECT
    id_processo,
    id_despacho,
    data_despacho,
    cancelado,
    tipo_motivacao,
    sigla_tipo_motivacao,
    year_snapshot
FROM exploded;


Running query in 'duck'

Count
7428


In [55]:
%%sql
DROP TABLE IF EXISTS outros_numeros_senado;

CREATE TABLE outros_numeros_senado AS
WITH base AS (
    SELECT
        TRY_CAST(payload_json AS JSON) AS j,
        year AS year_snapshot,
        CAST(json_extract_string(payload_json, '$.id') AS BIGINT) AS id_processo
    FROM bronze_senado_processo
),
exploded AS (
    SELECT
        b.id_processo,
        CAST(json_extract_string(o.value, '$.idOutroProcesso') AS BIGINT) AS id_outro_processo,
        CAST(json_extract_string(o.value, '$.ano') AS INTEGER) AS ano,
        json_extract_string(o.value, '$.casaIdentificadora') AS casa_identificadora,
        json_extract_string(o.value, '$.enteIdentificador') AS ente_identificador,
        json_extract_string(o.value, '$.sigla') AS sigla,
        json_extract_string(o.value, '$.numero') AS numero,
        json_extract_string(o.value, '$.siglaEnteIdentificador') AS sigla_ente_identificador,
        json_extract_string(o.value, '$.externaAoCongresso') AS externa_ao_congresso,
        json_extract_string(o.value, '$.tramitando') AS tramitando,
        b.year_snapshot
    FROM base b
    CROSS JOIN json_each(b.j, '$.outrosNumeros') AS o
),
numbered AS (
    SELECT
        ROW_NUMBER() OVER () AS id_outro_numero,
        id_processo,
        id_outro_processo,
        ano,
        casa_identificadora,
        ente_identificador,
        sigla,
        numero,
        sigla_ente_identificador,
        externa_ao_congresso,
        tramitando,
        year_snapshot
    FROM exploded
)
SELECT *
FROM numbered;

Running query in 'duck'

Count
4062


In [56]:
%%sql
DROP TABLE IF EXISTS processos_relacionados_senado;

CREATE TABLE processos_relacionados_senado AS
WITH base AS (
    SELECT
        TRY_CAST(payload_json AS JSON) AS j,
        year AS year_snapshot,
        CAST(json_extract_string(payload_json, '$.id') AS BIGINT) AS id_processo
    FROM bronze_senado_processo
),
exploded AS (
    SELECT
        b.id_processo,
        CAST(json_extract_string(p.value, '$.idOutroProcesso') AS BIGINT) AS id_outro_processo,
        CAST(json_extract_string(p.value, '$.ano') AS INTEGER) AS ano,
        json_extract_string(p.value, '$.casaIdentificadora') AS casa_identificadora,
        json_extract_string(p.value, '$.enteIdentificador') AS ente_identificador,
        json_extract_string(p.value, '$.sigla') AS sigla,
        json_extract_string(p.value, '$.numero') AS numero,
        json_extract_string(p.value, '$.siglaEnteIdentificador') AS sigla_ente_identificador,
        json_extract_string(p.value, '$.tipoRelacao') AS tipo_relacao,
        json_extract_string(p.value, '$.tramitando') AS tramitando,
        b.year_snapshot
    FROM base b
    CROSS JOIN json_each(b.j, '$.processosRelacionados') AS p
),
numbered AS (
    SELECT
        ROW_NUMBER() OVER () AS id_processo_relacionado,
        id_processo,
        id_outro_processo,
        ano,
        casa_identificadora,
        ente_identificador,
        sigla,
        numero,
        sigla_ente_identificador,
        tipo_relacao,
        tramitando,
        year_snapshot
    FROM exploded
)
SELECT *
FROM numbered;

Running query in 'duck'

Count
8363


In [57]:
%%sql
DROP TABLE IF EXISTS documento_autoria_senado;

CREATE TABLE documento_autoria_senado AS
WITH base AS (
    SELECT
        TRY_CAST(payload_json AS JSON) AS j,
        year AS year_snapshot,
        CAST(json_extract_string(payload_json, '$.id') AS BIGINT) AS id_processo
    FROM bronze_senado_processo
),
exploded AS (
    SELECT
        b.id_processo,
        CAST(json_extract_string(a.value, '$.idEnte') AS BIGINT) AS id_ente,
        json_extract_string(a.value, '$.autor') AS autor,
        json_extract_string(a.value, '$.casaEnte') AS casa_ente,
        CAST(json_extract_string(a.value, '$.codigoParlamentar') AS BIGINT) as codigo_parlamentar,
        json_extract_string(a.value, '$.descricaoTipo') AS descricao_tipo,
        json_extract_string(a.value, '$.ente') AS ente,
        CAST(json_extract_string(a.value, '$.ordem') AS INTEGER) AS ordem,
        json_extract_string(a.value, '$.outrosAutoresNaoInformados') AS outros_autores_nao_informados,
        json_extract_string(a.value, '$.siglaEnte') AS sigla_ente,
        json_extract_string(a.value, '$.siglaTipo') AS sigla_tipo,
        b.year_snapshot
    FROM base b
    CROSS JOIN json_each(b.j, '$.documento.autoria') AS a
),
numbered AS (
    SELECT
        ROW_NUMBER() OVER () AS id_documento_autoria,
        id_processo,
        id_ente,
        autor,
        codigo_parlamentar,
        descricao_tipo,
        ente,
        ordem,
        outros_autores_nao_informados,
        sigla_ente,
        sigla_tipo,
        year_snapshot
    FROM exploded
)
SELECT *
FROM numbered;


Running query in 'duck'

Count
36479


In [58]:
%%sql
DROP TABLE IF EXISTS autuacoes_senado;

CREATE TABLE autuacoes_senado AS
WITH base AS (
    SELECT
        TRY_CAST(payload_json AS JSON)                                   AS j,
        year                                                             AS year_snapshot,
        CAST(json_extract_string(payload_json, '$.id') AS BIGINT)        AS id_processo
    FROM bronze_senado_processo
),
exploded AS (
    SELECT
        b.id_processo,
        -- array position within $.autuacoes becomes a stable, deterministic PK within the processo
        CAST(a.key AS INTEGER) + 1                                       AS autuacao_idx,   -- 1-based
        json_extract_string(a.value, '$.descricao')                      AS descricao_autuacao,
        CAST(json_extract_string(a.value, '$.idEnteControleAtual') AS BIGINT)
                                                                          AS id_ente_controle_atual,
        json_extract_string(a.value, '$.nomeEnteControleAtual')          AS nome_ente_controle_atual,
        json_extract_string(a.value, '$.siglaEnteControleAtual')         AS sigla_ente_controle_atual,
        CAST(json_extract_string(a.value, '$.numero') AS INTEGER)        AS numero_autuacao,
        b.year_snapshot
    FROM base b
    CROSS JOIN json_each(b.j, '$.autuacoes') AS a
)
SELECT
    id_processo,
    autuacao_idx,
    descricao_autuacao,
    id_ente_controle_atual,
    nome_ente_controle_atual,
    sigla_ente_controle_atual,
    numero_autuacao,
    year_snapshot
FROM exploded;


Running query in 'duck'

Count
19816


In [59]:
%%sql
DROP TABLE IF EXISTS informes_legislativos_senado;

CREATE TABLE informes_legislativos_senado AS
WITH base AS (
    SELECT
        TRY_CAST(payload_json AS JSON)                            AS j,
        year                                                      AS year_snapshot,
        CAST(json_extract_string(payload_json, '$.id') AS BIGINT) AS id_processo
    FROM bronze_senado_processo
),
autuacoes AS (
    SELECT
        b.id_processo,
        b.year_snapshot,
        a.value AS autuacao_json
    FROM base b
    CROSS JOIN json_each(b.j, '$.autuacoes') AS a
),
informes AS (
    SELECT
        a.id_processo,
        a.year_snapshot,
        i.value AS informe_json
    FROM autuacoes a
    CROSS JOIN json_each(a.autuacao_json, '$.informesLegislativos') AS i
)
SELECT
    ROW_NUMBER() OVER ()                                         AS id_informe_legislativo, -- auto increment PK
    id_processo,

    CAST(json_extract_string(informe_json, '$.id') AS BIGINT)    AS id_informe,
    TRY_CAST(json_extract_string(informe_json, '$.data') AS DATE) AS data_informe,
    json_extract_string(informe_json, '$.descricao')             AS descricao,

    -- situação iniciada (optional)
    CAST(json_extract_string(informe_json, '$.idSituacaoIniciada') AS BIGINT) AS id_situacao_iniciada,
    json_extract_string(informe_json, '$.siglaSituacaoIniciada')  AS sigla_situacao_iniciada,

    -- enteAdministrativo
    json_extract_string(informe_json, '$.enteAdministrativo.casa')               AS ente_adm_casa,
    CAST(json_extract_string(informe_json, '$.enteAdministrativo.id') AS BIGINT) AS ente_adm_id,
    json_extract_string(informe_json, '$.enteAdministrativo.nome')               AS ente_adm_nome,
    json_extract_string(informe_json, '$.enteAdministrativo.sigla')              AS ente_adm_sigla,

    -- colegiado
    json_extract_string(informe_json, '$.colegiado.casa')          AS colegiado_casa,
    CAST(json_extract_string(informe_json, '$.colegiado.codigo') AS BIGINT) AS colegiado_codigo,
    json_extract_string(informe_json, '$.colegiado.nome')          AS colegiado_nome,
    json_extract_string(informe_json, '$.colegiado.sigla')         AS colegiado_sigla,

    year_snapshot
FROM informes;


Running query in 'duck'

Count
318312


In [60]:
%%sql
DROP TABLE IF EXISTS informes_documentos_associados_senado;

CREATE TABLE informes_documentos_associados_senado AS
WITH base AS (
    SELECT
        TRY_CAST(payload_json AS JSON) AS j,
        year AS year_snapshot,
        CAST(json_extract_string(payload_json, '$.id') AS BIGINT) AS id_processo
    FROM bronze_senado_processo
),
autuacoes AS (
    SELECT
        b.id_processo,
        CAST(a.key AS INTEGER)                    AS autuacao_ordem,
        a.value                                   AS autuacao_json,
        b.year_snapshot
    FROM base b
    CROSS JOIN json_each(b.j, '$.autuacoes') AS a
),
informes AS (
    SELECT
        a.id_processo,
        a.autuacao_ordem,
        CAST(i.key AS INTEGER)                    AS informe_ordem,
        CAST(json_extract_string(i.value, '$.id') AS BIGINT) AS id_informe,
        i.value                                   AS informe_json,
        a.year_snapshot
    FROM autuacoes a
    CROSS JOIN json_each(a.autuacao_json, '$.informesLegislativos') AS i
),
docs AS (
    SELECT
        inf.id_processo,
        inf.id_informe,
        inf.autuacao_ordem,
        inf.informe_ordem,
        CAST(d.key AS INTEGER)                    AS documento_ordem,
        CAST(json_extract_string(d.value, '$.id') AS BIGINT)   AS id_documento,
        json_extract_string(d.value, '$.siglaTipo')            AS sigla_tipo_documento,
        json_extract_string(d.value, '$.tipo')                 AS tipo_documento,
        json_extract_string(d.value, '$.identificacao')        AS identificacao,
        TRY_CAST(json_extract_string(d.value, '$.data') AS TIMESTAMP) AS data_documento,
        json_extract_string(d.value, '$.autoria')              AS autoria_documento,
        json_extract_string(d.value, '$.url')                  AS url_documento,
        inf.year_snapshot
    FROM informes inf
    CROSS JOIN json_each(inf.informe_json, '$.documentosAssociados') AS d
),
numbered AS (
    SELECT
        ROW_NUMBER() OVER () AS id_documento_associado,
        id_processo,
        id_informe,
        id_documento,
        autuacao_ordem,
        informe_ordem,
        documento_ordem,
        sigla_tipo_documento,
        tipo_documento,
        identificacao,
        data_documento,
        autoria_documento,
        url_documento,
        year_snapshot
    FROM docs
)
SELECT *
FROM numbered;


Running query in 'duck'

Count
92499


In [61]:
%%sql
-- Movimentações dentro de cada autuação do processo
DROP TABLE IF EXISTS movimentacoes_senado;

CREATE TABLE movimentacoes_senado AS
WITH base AS (
    SELECT
        TRY_CAST(payload_json AS JSON)                                           AS j,
        year                                                                     AS year_snapshot,
        CAST(json_extract_string(payload_json, '$.id') AS BIGINT)                AS id_processo
    FROM bronze_senado_processo
),
autuacoes AS (  -- explode top-level autuações and keep their array index as autuacao_idx
    SELECT
        b.id_processo,
        CAST(a.key AS INTEGER)                                                   AS autuacao_idx,
        a.value                                                                  AS autuacao_json,
        b.year_snapshot
    FROM base b
    CROSS JOIN json_each(b.j, '$.autuacoes') AS a
),
exploded AS (  -- explode movimentações under each autuação
    SELECT
        a.id_processo,
        a.autuacao_idx,
        CAST(m.key AS INTEGER)                                                   AS movimentacao_idx,
        CAST(json_extract_string(m.value, '$.id') AS BIGINT)                     AS id_movimentacao,
        TRY_CAST(json_extract_string(m.value, '$.dataEnvio') AS TIMESTAMP)       AS data_envio,
        TRY_CAST(json_extract_string(m.value, '$.dataRecebimento') AS TIMESTAMP) AS data_recebimento,

        -- Ente de origem
        json_extract_string(m.value, '$.enteOrigem.casa')                        AS ente_origem_casa,
        CAST(json_extract_string(m.value, '$.enteOrigem.id') AS BIGINT)          AS ente_origem_id,
        json_extract_string(m.value, '$.enteOrigem.nome')                        AS ente_origem_nome,
        json_extract_string(m.value, '$.enteOrigem.sigla')                       AS ente_origem_sigla,

        -- Ente de destino
        json_extract_string(m.value, '$.enteDestino.casa')                       AS ente_destino_casa,
        CAST(json_extract_string(m.value, '$.enteDestino.id') AS BIGINT)         AS ente_destino_id,
        json_extract_string(m.value, '$.enteDestino.nome')                       AS ente_destino_nome,
        json_extract_string(m.value, '$.enteDestino.sigla')                      AS ente_destino_sigla,

        -- Colegiado de destino (pode vir vazio {})
        json_extract_string(m.value, '$.colegiadoDestino.casa')                  AS colegiado_destino_casa,
        CAST(json_extract_string(m.value, '$.colegiadoDestino.codigo') AS BIGINT) AS colegiado_destino_codigo,
        json_extract_string(m.value, '$.colegiadoDestino.nome')                  AS colegiado_destino_nome,
        json_extract_string(m.value, '$.colegiadoDestino.sigla')                 AS colegiado_destino_sigla,

        a.year_snapshot
    FROM autuacoes a
    CROSS JOIN json_each(a.autuacao_json, '$.movimentacoes') AS m
)
SELECT
    id_processo,
    autuacao_idx,            -- FK para autuacoes_senado (id_processo, autuacao_idx)
    movimentacao_idx,        -- posição dentro da autuação (opcional como parte da PK composta)
    id_movimentacao,
    data_envio,
    data_recebimento,
    ente_origem_casa,
    ente_origem_id,
    ente_origem_nome,
    ente_origem_sigla,
    ente_destino_casa,
    ente_destino_id,
    ente_destino_nome,
    ente_destino_sigla,
    colegiado_destino_casa,
    colegiado_destino_codigo,
    colegiado_destino_nome,
    colegiado_destino_sigla,
    year_snapshot
FROM exploded;


Running query in 'duck'

Count
93583


In [62]:
%%sql
DROP TABLE IF EXISTS providencias_senado;

CREATE TABLE providencias_senado AS
WITH base AS (
  SELECT
    TRY_CAST(payload_json AS JSON) AS j,
    year AS year_snapshot,
    CAST(json_extract_string(payload_json, '$.id') AS BIGINT) AS id_processo
  FROM bronze_senado_processo
),
despachos AS (
  SELECT
    b.id_processo,
    CAST(json_extract_string(d.value, '$.id') AS BIGINT) AS id_despacho,
    d.value AS despacho_json,
    b.year_snapshot
  FROM base b
  CROSS JOIN json_each(b.j, '$.despachos') AS d
),
exploded AS (
  SELECT
    d.id_processo,
    d.id_despacho,
    CAST(json_extract_string(p.value, '$.id') AS BIGINT)            AS id_providencia,
    json_extract_string(p.value, '$.descricao')                      AS descricao,
    json_extract_string(p.value, '$.tipo')                           AS tipo,
    json_extract_string(p.value, '$.analiseConteudo')                AS analise_conteudo,
    json_extract_string(p.value, '$.analiseTempo')                   AS analise_tempo,
    CAST(json_extract_string(p.value, '$.ordem') AS INTEGER)         AS ordem,
    json_extract_string(p.value, '$.reexame')                        AS reexame,       -- "Sim"/"Não"
    d.year_snapshot
  FROM despachos d
  CROSS JOIN json_each(d.despacho_json, '$.providencias') AS p
)
SELECT
  id_processo,
  id_despacho,
  id_providencia,
  descricao,
  tipo,
  analise_conteudo,
  analise_tempo,
  ordem,
  reexame,
  year_snapshot
FROM exploded;


Running query in 'duck'

Count
7744


In [63]:
%%sql
DROP TABLE IF EXISTS unidades_destinatarias_senado;

CREATE TABLE unidades_destinatarias_senado AS
WITH base AS (
  SELECT
    TRY_CAST(payload_json AS JSON) AS j,
    year AS year_snapshot,
    CAST(json_extract_string(payload_json, '$.id') AS BIGINT) AS id_processo
  FROM bronze_senado_processo
),
desp AS (
  SELECT
    b.id_processo,
    b.year_snapshot,
    CAST(json_extract_string(d.value, '$.id') AS BIGINT) AS id_despacho,
    d.value AS despacho_json
  FROM base b
  CROSS JOIN json_each(b.j, '$.despachos') AS d
),
prov AS (
  SELECT
    d.id_processo,
    d.year_snapshot,
    d.id_despacho,
    CAST(json_extract_string(p.value, '$.id') AS BIGINT) AS id_providencia,
    p.value AS providencia_json
  FROM desp d
  CROSS JOIN json_each(d.despacho_json, '$.providencias') AS p
),
unid AS (
  SELECT
    p.id_processo,
    p.year_snapshot,
    p.id_despacho,
    p.id_providencia,
    CAST(json_extract_string(u.value, '$.colegiado.codigo') AS BIGINT) AS colegiado_codigo,
    json_extract_string(u.value, '$.colegiado.casa')  AS colegiado_casa,
    json_extract_string(u.value, '$.colegiado.nome')  AS colegiado_nome,
    json_extract_string(u.value, '$.colegiado.sigla') AS colegiado_sigla,
    CAST(json_extract_string(u.value, '$.ordem') AS INTEGER) AS ordem,
    json_extract_string(u.value, '$.tipoAnaliseDeliberacao') AS tipo_analise_deliberacao
  FROM prov p
  CROSS JOIN json_each(p.providencia_json, '$.unidadesDestinatarias') AS u
),
numbered AS (
  SELECT
    ROW_NUMBER() OVER () AS id_unidade_destinataria,
    id_processo,
    id_despacho,
    id_providencia,
    colegiado_casa,
    colegiado_codigo,
    colegiado_nome,
    colegiado_sigla,
    ordem,
    tipo_analise_deliberacao,
    year_snapshot
  FROM unid
)
SELECT *
FROM numbered;


Running query in 'duck'

Count
9667


In [ ]:
%%sql
DROP TABLE IF EXISTS encontro_legislativo_senado;

CREATE TABLE encontro_legislativo_senado AS
WITH base AS (
  SELECT
    TRY_CAST(payload_json AS JSON) AS j,
    year AS year_snapshot,
    CAST(json_extract_string(payload_json, '$.id') AS BIGINT) AS id_processo
  FROM bronze_senado_processo
),
desp AS (
  SELECT
    b.id_processo,
    CAST(json_extract_string(d.value, '$.id') AS BIGINT) AS id_despacho,
    d.value AS despacho_json,
    b.year_snapshot
  FROM base b
  CROSS JOIN json_each(b.j, '$.despachos') AS d
),
encontro AS (
  SELECT
    id_processo,
    id_despacho,
    CAST(json_extract_string(despacho_json, '$.encontroLegislativo.id') AS BIGINT)          AS id_encontro_legislativo,
    TRY_CAST(json_extract_string(despacho_json, '$.encontroLegislativo.data') AS DATE)      AS data_encontro,
    json_extract_string(despacho_json, '$.encontroLegislativo.tipo')                         AS tipo_encontro,
    json_extract_string(despacho_json, '$.encontroLegislativo.descricao')                    AS descricao_encontro,
    json_extract_string(despacho_json, '$.encontroLegislativo.casa')                         AS casa_encontro,
    CAST(json_extract_string(despacho_json, '$.encontroLegislativo.numero') AS INTEGER)      AS numero_encontro,
    json_extract_string(despacho_json, '$.encontroLegislativo.colegiado.casa')               AS colegiado_casa,
    CAST(json_extract_string(despacho_json, '$.encontroLegislativo.colegiado.codigo') AS BIGINT) AS colegiado_codigo,
    json_extract_string(despacho_json, '$.encontroLegislativo.colegiado.nome')               AS colegiado_nome,
    json_extract_string(despacho_json, '$.encontroLegislativo.colegiado.sigla')              AS colegiado_sigla,
    year_snapshot
  FROM desp
)
SELECT
  id_processo,
  id_despacho,
  id_encontro_legislativo,
  data_encontro,
  tipo_encontro,
  descricao_encontro,
  casa_encontro,
  numero_encontro,
  colegiado_casa,
  colegiado_codigo,
  colegiado_nome,
  colegiado_sigla,
  year_snapshot
FROM encontro
WHERE id_encontro_legislativo IS NOT NULL;


Running query in 'duck'

Count
4616


: 